In [1]:
"""
Ethereum Price Prediction - Data Loading & Preprocessing
"""
# ============================================================================
# 기본 라이브러리 및 유틸리티
# ============================================================================
import gc
import json
import joblib
import os
import warnings
from dotenv import load_dotenv
# 날짜/시간
from datetime import datetime, timedelta

# 데이터 처리
import numpy as np
import pandas as pd
import pandas_ta as ta
from collections import Counter

# ============================================================================
# ML/DL 라이브러리 및 도구
# ============================================================================

# 하이퍼파라미터 최적화
import optuna

# Scikit-learn: 데이터 전처리
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler

# Scikit-learn: 모델
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Scikit-learn: 앙상블 모델
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier, # 추가된 항목
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# Scikit-learn: 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, # 분류 지표
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error # 회귀 지표
)


from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping 
from xgboost import XGBClassifier, XGBRegressor

# TensorFlow/Keras 딥러닝
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    # 기본 레이어
    Input, Dense, Flatten, Dropout, Activation,
    # RNN 레이어
    LSTM, GRU, SimpleRNN, Bidirectional,
    # CNN 레이어
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    # 정규화 레이어
    BatchNormalization, LayerNormalization,
    # Attention 레이어
    Attention, MultiHeadAttention,
    # 유틸리티 레이어
    Concatenate, Add, Multiply, Lambda,
    Reshape, Permute, RepeatVector, TimeDistributed
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 시계열 분석 (Statsmodels)
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# PyTorch 
try:
    import torch
    import torch.nn as nn
except ImportError:
    pass




import optuna
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, BaggingClassifier,
    GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier,
    VotingClassifier, HistGradientBoostingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# ============================================================================
# 환경 설정 및 경고 무시
# ============================================================================

# GPU 메모리 증가 허용 설정
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

warnings.filterwarnings('ignore')


DATA_DIR_MAIN = './macro_data'
DATA_DIR_NEW = './macro_data/macro_data'

TRAIN_START_DATE = pd.to_datetime('2020-01-01')
LOOKBACK_DAYS = 200
LOOKBACK_START_DATE = TRAIN_START_DATE - timedelta(days=LOOKBACK_DAYS)


def standardize_date_column(df,file_name):
    """날짜 컬럼 자동 탐지 + datetime 통일 + tz 제거 + 시각 제거"""

    date_cols = [col for col in df.columns if 'date' in col.lower()]
    if not date_cols:
        print("[Warning] 날짜 컬럼을 찾을 수 없습니다.")
        return df
    date_col = date_cols[0]
    
    if date_col != 'date':
        df.rename(columns={date_col: 'date'}, inplace=True)
    

    if file_name == 'eth_onchain.csv':
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)
    
    df = df.dropna(subset=['date'])
    df['date'] = df['date'].dt.normalize()  
    if pd.api.types.is_datetime64tz_dtype(df['date']):
        df['date'] = df['date'].dt.tz_convert(None)
    else:
        df['date'] = df['date'].dt.tz_localize(None)
    print(df.shape,file_name)
    return df


def load_csv(directory, filename):
    filepath = os.path.join(directory, filename)
    if not os.path.exists(filepath):
        print(f"[Warning] {filename} not found")
        return pd.DataFrame()
    df = pd.read_csv(filepath)
    return standardize_date_column(df, filename)


def add_prefix(df, prefix):
    if df.empty:
        return df
    df.columns = [f"{prefix}_{col}" if col != 'date' else col for col in df.columns]
    return df


def create_sentiment_features(news_df):
    if news_df.empty:
        return pd.DataFrame(columns=['date'])
    
    agg = news_df.groupby('date').agg(
        sentiment_mean=('label', 'mean'),
        sentiment_std=('label', 'std'),
        news_count=('label', 'count'),
        positive_ratio=('label', lambda x: (x == 1).sum() / len(x)),
        negative_ratio=('label', lambda x: (x == -1).sum() / len(x)),
        extreme_positive_count=('label', lambda x: (x == 1).sum()),
        extreme_negative_count=('label', lambda x: (x == -1).sum()),
        sentiment_sum=('label', 'sum'),
    ).reset_index().fillna(0)
    
    agg['sentiment_polarity'] = agg['positive_ratio'] - agg['negative_ratio']
    agg['sentiment_intensity'] = agg['positive_ratio'] + agg['negative_ratio']
    agg['sentiment_disagreement'] = agg['positive_ratio'] * agg['negative_ratio']
    agg['bull_bear_ratio'] = agg['positive_ratio'] / (agg['negative_ratio'] + 1e-10)
    agg['weighted_sentiment'] = agg['sentiment_mean'] * np.log1p(agg['news_count'])
    agg['extremity_index'] = (agg['extreme_positive_count'] + agg['extreme_negative_count']) / (agg['news_count'] + 1e-10)
    
    for window in [3,7]:
        agg[f'sentiment_ma{window}'] = agg['sentiment_mean'].rolling(window=window, min_periods=1).mean()
        agg[f'sentiment_volatility_{window}'] = agg['sentiment_mean'].rolling(window=window, min_periods=1).std()
    
    agg['sentiment_trend'] = agg['sentiment_mean'].diff()
    agg['sentiment_acceleration'] = agg['sentiment_trend'].diff()
    agg['news_volume_change'] = agg['news_count'].pct_change()
    
    for window in [7, 14]:
        agg[f'news_volume_ma{window}'] = agg['news_count'].rolling(window=window, min_periods=1).mean()
    
    return agg.fillna(0)


def smart_fill_missing(df_merged):
    REFERENCE_START_DATE = pd.to_datetime('2020-01-01')
    
    for col in df_merged.columns:
        if col == 'date':
            continue
        
        if df_merged[col].isnull().sum() == 0:
            continue
        
        non_null_idx = df_merged[col].first_valid_index()
        
        if non_null_idx is None:
            df_merged[col] = df_merged[col].fillna(0)
            continue
        
        first_date = df_merged.loc[non_null_idx, 'date']
        
        before_mask = df_merged['date'] < first_date
        after_mask = df_merged['date'] >= first_date
        
        df_merged.loc[before_mask, col] = df_merged.loc[before_mask, col].fillna(0)
        df_merged.loc[after_mask, col] = df_merged.loc[after_mask, col].fillna(method='ffill')
        
        remaining = df_merged.loc[after_mask, col].isnull().sum()
        if remaining > 0:
            df_merged.loc[after_mask, col] = df_merged.loc[after_mask, col].fillna(0)
    
    return df_merged


print("="*80)
print("DATA LOADING")
print("="*80)

news_df = load_csv(DATA_DIR_MAIN, 'news_data.csv')
eth_onchain_df = load_csv(DATA_DIR_MAIN, 'eth_onchain.csv')
macro_df = load_csv(DATA_DIR_NEW, 'macro_crypto_data.csv')
sp500_df = load_csv(DATA_DIR_NEW, 'SP500.csv')
vix_df = load_csv(DATA_DIR_NEW, 'VIX.csv')
gold_df = load_csv(DATA_DIR_NEW, 'GOLD.csv')
dxy_df = load_csv(DATA_DIR_NEW, 'DXY.csv')
fear_greed_df = load_csv(DATA_DIR_NEW, 'fear_greed.csv')
eth_funding_df = load_csv(DATA_DIR_NEW, 'eth_funding_rate.csv')
usdt_eth_mcap_df = load_csv(DATA_DIR_NEW, 'usdt_eth_mcap.csv')
aave_tvl_df = load_csv(DATA_DIR_NEW, 'aave_eth_tvl.csv')
lido_tvl_df = load_csv(DATA_DIR_NEW, 'lido_eth_tvl.csv')
makerdao_tvl_df = load_csv(DATA_DIR_NEW, 'makerdao_eth_tvl.csv')
uniswap_tvl_df = load_csv(DATA_DIR_NEW, 'uniswap_eth_tvl.csv')
curve_tvl_df = load_csv(DATA_DIR_NEW, 'curve-dex_eth_tvl.csv')
eth_chain_tvl_df = load_csv(DATA_DIR_NEW, 'eth_chain_tvl.csv')
layer2_tvl_df = load_csv(DATA_DIR_NEW, 'layer2_tvl.csv')

print(f"Loaded {len([df for df in [fear_greed_df, eth_funding_df, usdt_eth_mcap_df, aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df, eth_chain_tvl_df, layer2_tvl_df] if not df.empty])} files")

all_dataframes = [
    macro_df, news_df, eth_onchain_df, fear_greed_df, usdt_eth_mcap_df,
    aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df,
    eth_chain_tvl_df, eth_funding_df, layer2_tvl_df, 
    sp500_df, vix_df, gold_df, dxy_df
]

last_dates = [
    pd.to_datetime(df['date']).max() 
    for df in all_dataframes 
    if not df.empty and 'date' in df.columns
]

end_date = min(last_dates) if last_dates else pd.Timestamp.today()

print("\n" + "="*80)
print("SENTIMENT FEATURES")
print("="*80)
sentiment_features = create_sentiment_features(news_df)
print(f"Generated {sentiment_features.shape[1]-1} features")

print("\n" + "="*80)
print("DATA MERGING")
print("="*80)

eth_onchain_df = add_prefix(eth_onchain_df, 'eth')
fear_greed_df = add_prefix(fear_greed_df, 'fg')
usdt_eth_mcap_df = add_prefix(usdt_eth_mcap_df, 'usdt')
aave_tvl_df = add_prefix(aave_tvl_df, 'aave')
lido_tvl_df = add_prefix(lido_tvl_df, 'lido')
makerdao_tvl_df = add_prefix(makerdao_tvl_df, 'makerdao')
uniswap_tvl_df = add_prefix(uniswap_tvl_df, 'uniswap')
curve_tvl_df = add_prefix(curve_tvl_df, 'curve')
eth_chain_tvl_df = add_prefix(eth_chain_tvl_df, 'chain')
eth_funding_df = add_prefix(eth_funding_df, 'funding')
layer2_tvl_df = add_prefix(layer2_tvl_df, 'l2')
sp500_df = add_prefix(sp500_df, 'sp500')
vix_df = add_prefix(vix_df, 'vix')
gold_df = add_prefix(gold_df, 'gold')
dxy_df = add_prefix(dxy_df, 'dxy')

date_range = pd.date_range(start=LOOKBACK_START_DATE, end=end_date, freq='D')
df_merged = pd.DataFrame(date_range, columns=['date'])

dataframes_to_merge = [
    macro_df, sentiment_features, eth_onchain_df, fear_greed_df, usdt_eth_mcap_df,
    aave_tvl_df, lido_tvl_df, makerdao_tvl_df, uniswap_tvl_df, curve_tvl_df,
    eth_chain_tvl_df, eth_funding_df, layer2_tvl_df,
    sp500_df, vix_df, gold_df, dxy_df
]

for df in dataframes_to_merge:
    if not df.empty:
        df_merged = pd.merge(df_merged, df, on='date', how='left')

print(f"Merged shape: {df_merged.shape}")
print(f"Missing before fill: {df_merged.isnull().sum().sum():,}")

print("\n" + "="*80)
print("MISSING VALUE HANDLING")
print("="*80)

df_merged = smart_fill_missing(df_merged)

missing_after = df_merged.isnull().sum().sum()
print(f"Missing after fill: {missing_after:,}")

if missing_after > 0:
    df_merged = df_merged.fillna(0)
    print(f"Remaining filled with 0")

lookback_df = df_merged[df_merged['date'] < TRAIN_START_DATE]
cols_to_drop = [
    col for col in lookback_df.columns 
    if lookback_df[col].isnull().all() and col != 'date'
]

if cols_to_drop:
    print(f"\nDropping {len(cols_to_drop)} fully missing columns")
    df_merged = df_merged.drop(columns=cols_to_drop)

print(f"Shape: {df_merged.shape}")
print(f"Period: {df_merged['date'].min().date()} ~ {df_merged['date'].max().date()}")
print(f"Missing: {df_merged.isnull().sum().sum()}")

print(f"\nFeature groups:")
print(f"  Crypto prices: {len([c for c in df_merged.columns if any(x in c for x in ['BTC_', 'ETH_', 'BNB_', 'XRP_', 'SOL_', 'ADA_', 'DOGE_', 'AVAX_', 'DOT_'])])}")
print(f"  On-chain: {len([c for c in df_merged.columns if c.startswith('eth_')])}")
print(f"  DeFi TVL: {len([c for c in df_merged.columns if any(x in c for x in ['aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_'])])}")
print(f"  Layer 2: {len([c for c in df_merged.columns if c.startswith('l2_')])}")
print(f"  Sentiment: {len([c for c in df_merged.columns if any(x in c for x in ['sentiment', 'news', 'bull_bear', 'positive', 'negative', 'extreme'])])}")
print(f"  Macro: {len([c for c in df_merged.columns if any(x in c for x in ['sp500_', 'vix_', 'gold_', 'dxy_'])])}")
print(f"  Fear & Greed: {len([c for c in df_merged.columns if c.startswith('fg_')])}")
print(f"  Funding Rate: {len([c for c in df_merged.columns if c.startswith('funding_')])}")
print(f"  Stablecoin: {len([c for c in df_merged.columns if c.startswith('usdt_')])}")
print("="*80)



2025-10-29 22:38:17.751834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-29 22:38:17.751873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-29 22:38:17.753481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-29 22:38:17.761154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 22:38:18.606750: W tensorflow/comp

DATA LOADING
(26212, 3) news_data.csv
(3732, 11) eth_onchain.csv
(3224, 46) macro_crypto_data.csv
(2218, 2) SP500.csv
(2218, 2) VIX.csv
(2219, 2) GOLD.csv
(2220, 2) DXY.csv
(2824, 2) fear_greed.csv
(2164, 2) eth_funding_rate.csv
(2892, 6) usdt_eth_mcap.csv
(1990, 2) aave_eth_tvl.csv
(1776, 2) lido_eth_tvl.csv
(2492, 2) makerdao_eth_tvl.csv
(2549, 2) uniswap_eth_tvl.csv
(2084, 2) curve-dex_eth_tvl.csv
(2955, 2) eth_chain_tvl.csv
(1582, 5) layer2_tvl.csv
Loaded 10 files

SENTIMENT FEATURES
Generated 23 features

DATA MERGING
Merged shape: (2325, 100)
Missing before fill: 20,652

MISSING VALUE HANDLING
Missing after fill: 0
Shape: (2325, 100)
Period: 2019-06-15 ~ 2025-10-24
Missing: 0

Feature groups:
  Crypto prices: 45
  On-chain: 10
  DeFi TVL: 6
  Layer 2: 4
  Sentiment: 22
  Macro: 4
  Fear & Greed: 1
  Funding Rate: 1
  Stablecoin: 5


In [2]:
eth_onchain_df.head(10)

,date,eth_tx_count,eth_active_addresses,eth_new_addresses,eth_large_eth_transfers,eth_token_transfers,eth_contract_events,eth_avg_gas_price,eth_total_gas_used,eth_avg_block_size,eth_avg_block_difficulty
0,2015-08-07,2050,784,784,283,0,0,6.046842e+11,49353826,632.63,1.470839e+12
1,2015-08-08,2881,605,430,186,0,6,3.227136e+11,376006093,667.59,1.586124e+12
2,2015-08-09,1329,462,252,124,0,11,4.754671e+11,38863003,618.30,1.709480e+12
3,2015-08-10,2037,821,632,115,0,22,4.216549e+11,74070061,631.19,1.837696e+12
4,2015-08-11,4963,2132,1881,150,0,42,7.783882e+10,163481740,692.01,2.036391e+12
5,2015-08-12,2036,581,259,118,0,111,4.449024e+11,70102332,653.43,2.207080e+12
6,2015-08-13,2842,872,452,176,0,136,2.686835e+11,88234087,665.72,2.336980e+12
7,2015-08-14,3174,1115,562,265,0,81,1.934555e+11,78746522,659.75,2.671253e+12
8,2015-08-15,2284,857,294,179,0,61,1.443689e+11,59565914,638.34,3.378028e+12
9,2015-08-16,2440,765,202,165,0,59,1.209401e+11,58241191,640.65,3.631632e+12


In [2]:
def add_indicator_to_df(df_ta, indicator):
    """pandas_ta 지표 결과를 DataFrame에 안전하게 추가"""
    if indicator is None:
        return

    if isinstance(indicator, pd.DataFrame) and not indicator.empty:
        for col in indicator.columns:
            df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series) and not indicator.empty:
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    """지표 생성 시 오류 방지를 위한 래퍼 함수"""
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except Exception as e:
        func_name = func.__name__ if hasattr(func, '__name__') else str(func)
        print(f"    ⚠ {func_name.upper()} 생성 실패: {str(e)[:50]}")
        return False

def calculate_technical_indicators(df):
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    open_ = df.get('ETH_Open', close)

    try:
        # ===== MOMENTUM INDICATORS =====
        
        # RSI (14만 - 모든 fold 선택)
        df_ta['RSI_14'] = ta.rsi(close, length=14)
        
        # MACD (필수 - 자주 선택됨)
        safe_add(df_ta, ta.macd, close, fast=12, slow=26, signal=9)
        
        # Stochastic (14만 - 나머지는 중복)
        safe_add(df_ta, ta.stoch, high, low, close, k=14, d=3)
        
        # Williams %R
        df_ta['WILLR_14'] = ta.willr(high, low, close, length=14)
        
        # ROC (10만 - 20과 거의 동일)
        df_ta['ROC_10'] = ta.roc(close, length=10)
        
        # MOM (10만 유지)
        df_ta['MOM_10'] = ta.mom(close, length=10)
        
        # CCI (14, 50만 - 극단값 비교용)
        df_ta['CCI_14'] = ta.cci(high, low, close, length=14)
        df_ta['CCI_50'] = ta.cci(high, low, close, length=50)
        df_ta['CCI_SIGNAL'] = (df_ta['CCI_14'] > 100).astype(int)
      
        # TSI
        safe_add(df_ta, ta.tsi, close, fast=13, slow=25, signal=13)
        
        # Ichimoku (유지 - 복합 지표로 유용)
        try:
            ichimoku = ta.ichimoku(high, low, close)
            if ichimoku is not None and isinstance(ichimoku, tuple):
                ichimoku_df = ichimoku[0]
                if ichimoku_df is not None:
                    for col in ichimoku_df.columns:
                        df_ta[col] = ichimoku_df[col]
        except:
            pass

        # ===== OVERLAP INDICATORS =====
        
        # SMA (20, 50만 - Golden Cross용)
        df_ta['SMA_20'] = ta.sma(close, length=20)
        df_ta['SMA_50'] = ta.sma(close, length=50)
        
        # EMA (12, 26만 - MACD 구성 요소)
        df_ta['EMA_12'] = ta.ema(close, length=12)
        df_ta['EMA_26'] = ta.ema(close, length=26)
        
        # TEMA (10만 - 30과 중복)
        df_ta['TEMA_10'] = ta.tema(close, length=10)
        
        # WMA (20만 - 10과 중복)
        df_ta['WMA_20'] = ta.wma(close, length=20)
        
        # HMA (유지 - 독특한 smoothing)
        df_ta['HMA_9'] = ta.hma(close, length=9)
        
        # DEMA (유지)
        df_ta['DEMA_10'] = ta.dema(close, length=10)
        
        # VWMA (유지 - 거래량 가중)
        df_ta['VWMA_20'] = ta.vwma(close, volume, length=20)
        
        # 가격 조합 (유지 - 다른 정보)
        df_ta['HL2'] = ta.hl2(high, low)
        df_ta['HLC3'] = ta.hlc3(high, low, close)
        df_ta['OHLC4'] = ta.ohlc4(open_, high, low, close)

        # ===== VOLATILITY INDICATORS =====
        
        # Bollinger Bands 
        safe_add(df_ta, ta.bbands, close, length=20, std=2)
        
        # ATR 
        df_ta['ATR_14'] = ta.atr(high, low, close, length=14)
        
        # NATR
        df_ta['NATR_14'] = ta.natr(high, low, close, length=14)
        
        # True Range
        try:
            tr = ta.true_range(high, low, close)
            if isinstance(tr, pd.Series) and not tr.empty:
                df_ta['TRUERANGE'] = tr
            elif isinstance(tr, pd.DataFrame) and not tr.empty:
                df_ta['TRUERANGE'] = tr.iloc[:, 0]
        except:
            pass
        
        # Keltner Channel
        safe_add(df_ta, ta.kc, high, low, close, length=20)
        
        # Donchian Channel
        try:
            dc = ta.donchian(high, low, lower_length=20, upper_length=20)
            if dc is not None and isinstance(dc, pd.DataFrame) and not dc.empty:
                for col in dc.columns:
                    df_ta[col] = dc[col]
        except:
            pass
        
        # Supertrend
        atr_10 = ta.atr(high, low, close, length=10)
        hl2_calc = (high + low) / 2
        upper_band = hl2_calc + (3 * atr_10)
        lower_band = hl2_calc - (3 * atr_10)
        
        df_ta['SUPERTREND'] = 0
        for i in range(1, len(df_ta)):
            if close.iloc[i] > upper_band.iloc[i-1]:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = 1
            elif close.iloc[i] < lower_band.iloc[i-1]:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = -1
            else:
                df_ta.loc[df_ta.index[i], 'SUPERTREND'] = df_ta['SUPERTREND'].iloc[i-1]

        # ===== VOLUME INDICATORS =====
        
        # OBV (필수)
        df_ta['OBV'] = ta.obv(close, volume)
        
        # AD
        df_ta['AD'] = ta.ad(high, low, close, volume)
        
        # ADOSC
        df_ta['ADOSC_3_10'] = ta.adosc(high, low, close, volume, fast=3, slow=10)
        
        # MFI
        df_ta['MFI_14'] = ta.mfi(high, low, close, volume, length=14)
        
        # CMF
        df_ta['CMF_20'] = ta.cmf(high, low, close, volume, length=20)
        
        # EFI (Fold에서 선택됨)
        df_ta['EFI_13'] = ta.efi(close, volume, length=13)
        
        # EOM
        safe_add(df_ta, ta.eom, high, low, close, volume, length=14)
        
        # VWAP
        try:
            df_ta['VWAP'] = ta.vwap(high, low, close, volume)
        except:
            pass

        # ===== TREND INDICATORS =====
        
        # ADX (필수)
        safe_add(df_ta, ta.adx, high, low, close, length=14)
        
        # Aroon
        try:
            aroon = ta.aroon(high, low, length=25)
            if aroon is not None and isinstance(aroon, pd.DataFrame):
                for col in aroon.columns:
                    df_ta[col] = aroon[col]
        except:
            pass
        
        # PSAR
        try:
            psar = ta.psar(high, low, close)
            if psar is not None:
                if isinstance(psar, pd.DataFrame) and not psar.empty:
                    for col in psar.columns:
                        df_ta[col] = psar[col]
                elif isinstance(psar, pd.Series) and not psar.empty:
                    df_ta[psar.name] = psar
        except:
            pass
        
        # Vortex 
        safe_add(df_ta, ta.vortex, high, low, close, length=14)
        
        # DPO 
        df_ta['DPO_20'] = ta.dpo(close, length=20)

        # ===== 파생 지표 =====
        
        # 가격 변화율 
        df_ta['PRICE_CHANGE'] = close.pct_change()
        
        # 변동성 
        df_ta['VOLATILITY_20'] = close.pct_change().rolling(window=20).std()
        
        # 모멘텀 
        df_ta['MOMENTUM_10'] = close / close.shift(10) - 1
        
        # 이동평균 대비 위치 
        df_ta['PRICE_VS_SMA20'] = close / df_ta['SMA_20'] - 1
        df_ta['PRICE_VS_EMA12'] = close / df_ta['EMA_12'] - 1
        
        # 크로스 신호 
        df_ta['SMA_GOLDEN_CROSS'] = (df_ta['SMA_50'] > df_ta['SMA_20']).astype(int)
        df_ta['EMA_CROSS_SIGNAL'] = (df_ta['EMA_12'] > df_ta['EMA_26']).astype(int)
        
        # 거래량 지표
        df_ta['VOLUME_SMA_20'] = ta.sma(volume, length=20)
        df_ta['VOLUME_RATIO'] = volume / (df_ta['VOLUME_SMA_20'] + 1e-10)
        df_ta['VOLUME_CHANGE'] = volume.pct_change()
        df_ta['VOLUME_CHANGE_5'] = volume.pct_change(periods=5)
        
        # Range 지표 
        df_ta['HIGH_LOW_RANGE'] = (high - low) / (close + 1e-10)
        df_ta['HIGH_CLOSE_RANGE'] = np.abs(high - close.shift()) / (close + 1e-10)
        df_ta['CLOSE_LOW_RANGE'] = (close - low) / (close + 1e-10)
        
        # 일중 가격 위치
        df_ta['INTRADAY_POSITION'] = (close - low) / ((high - low) + 1e-10)
        
        # Linear Regression Slope 
        try:
            df_ta['SLOPE_5'] = ta.linreg(close, length=5, slope=True)
        except:
            df_ta['SLOPE_5'] = close.rolling(window=5).apply(
                lambda x: np.polyfit(np.arange(len(x)), x, 1)[0] if len(x) == 5 else np.nan, raw=True
            )
        
        # Increasing 
        df_ta['INC_1'] = (close > close.shift(1)).astype(int)
        
        # BOP
        df_ta['BOP'] = (close - open_) / ((high - low) + 1e-10)
        df_ta['BOP'] = df_ta['BOP'].fillna(0)
        
        # ===== 고급 파생 지표 =====
        
        # Bollinger Bands 파생 
        if 'BBL_20' in df_ta.columns and 'BBU_20' in df_ta.columns and 'BBM_20' in df_ta.columns:
            df_ta['BB_WIDTH'] = (df_ta['BBU_20'] - df_ta['BBL_20']) / (df_ta['BBM_20'] + 1e-8)
            df_ta['BB_POSITION'] = (close - df_ta['BBL_20']) / (df_ta['BBU_20'] - df_ta['BBL_20'] + 1e-8)
        
        # RSI 파생
        df_ta['RSI_OVERBOUGHT'] = (df_ta['RSI_14'] > 70).astype(int)
        df_ta['RSI_OVERSOLD'] = (df_ta['RSI_14'] < 30).astype(int)
        
        # MACD 히스토그램 변화율
        if 'MACDh_12_26_9' in df_ta.columns:
            df_ta['MACD_HIST_CHANGE'] = df_ta['MACDh_12_26_9'].diff()
        
        # Volume Profile
        df_ta['VOLUME_STRENGTH'] = volume / volume.rolling(window=50).mean()
        
        # Price Acceleration
        df_ta['PRICE_ACCELERATION'] = close.pct_change().diff()
        
        # Gap (Fold에서 선택됨)
        df_ta['GAP'] = (open_ - close.shift(1)) / (close.shift(1) + 1e-10)
        
        # Distance from High/Low 
        df_ta['ROLLING_MAX_20'] = close.rolling(window=20).max()
        df_ta['ROLLING_MIN_20'] = close.rolling(window=20).min()
        df_ta['DISTANCE_FROM_HIGH'] = (df_ta['ROLLING_MAX_20'] - close) / (df_ta['ROLLING_MAX_20'] + 1e-10)
        df_ta['DISTANCE_FROM_LOW'] = (close - df_ta['ROLLING_MIN_20']) / (close + 1e-10)

        # Realized Volatility 
        ret_squared = close.pct_change() ** 2
        df_ta['RV_5'] = ret_squared.rolling(5).sum()
        df_ta['RV_20'] = ret_squared.rolling(20).sum()
        df_ta['RV_RATIO'] = df_ta['RV_5'] / (df_ta['RV_20'] + 1e-10)
        
        added = df_ta.shape[1] - df.shape[1]

    except Exception as e:
        print(f"\n❌ Error: {e}")

    return df_ta



def add_enhanced_cross_crypto_features(df):
    df_enhanced = df.copy()
    df_enhanced['eth_return'] = df['ETH_Close'].pct_change()
    df_enhanced['btc_return'] = df['BTC_Close'].pct_change()

    for lag in [1, 5]:
        df_enhanced[f'btc_return_lag{lag}'] = df_enhanced['btc_return'].shift(lag)

    for window in [3, 7, 14, 30, 60]:
        df_enhanced[f'eth_btc_corr_{window}d'] = (
            df_enhanced['eth_return'].rolling(window).corr(df_enhanced['btc_return'])
        )

    eth_vol = df_enhanced['eth_return'].abs()
    btc_vol = df_enhanced['btc_return'].abs()

    for window in [7, 14, 30]:
        df_enhanced[f'eth_btc_volcorr_{window}d'] = eth_vol.rolling(window).corr(btc_vol)
        df_enhanced[f'eth_btc_volcorr_sq_{window}d'] = (
            (df_enhanced['eth_return']**2).rolling(window).corr(df_enhanced['btc_return']**2)
        )

    df_enhanced['btc_eth_strength_ratio'] = (
        df_enhanced['btc_return'] / (df_enhanced['eth_return'].abs() + 1e-8)
    )
    df_enhanced['btc_eth_strength_ratio_7d'] = df_enhanced['btc_eth_strength_ratio'].rolling(7).mean()

    alt_returns = []
    for coin in ['BNB', 'XRP', 'SOL', 'ADA']:
        if f'{coin}_Close' in df.columns:
            alt_returns.append(df[f'{coin}_Close'].pct_change())

    if alt_returns:
        market_return = pd.concat(
            alt_returns + [df_enhanced['eth_return'], df_enhanced['btc_return']], axis=1
        ).mean(axis=1)
        df_enhanced['btc_dominance'] = df_enhanced['btc_return'] / (market_return + 1e-8)

    for window in [30, 60, 90]:
        covariance = df_enhanced['eth_return'].rolling(window).cov(df_enhanced['btc_return'])
        btc_variance = df_enhanced['btc_return'].rolling(window).var()
        df_enhanced[f'eth_btc_beta_{window}d'] = covariance / (btc_variance + 1e-8)

    df_enhanced['eth_btc_spread'] = df_enhanced['eth_return'] - df_enhanced['btc_return']
    df_enhanced['eth_btc_spread_ma7'] = df_enhanced['eth_btc_spread'].rolling(7).mean()
    df_enhanced['eth_btc_spread_std7'] = df_enhanced['eth_btc_spread'].rolling(7).std()

    btc_vol_ma = btc_vol.rolling(30).mean()
    high_vol_mask = btc_vol > btc_vol_ma
    df_enhanced['eth_btc_corr_highvol'] = np.nan
    df_enhanced['eth_btc_corr_lowvol'] = np.nan

    for i in range(30, len(df_enhanced)):
        window_data = df_enhanced.iloc[i-30:i]
        high_vol_data = window_data[high_vol_mask.iloc[i-30:i]]
        low_vol_data = window_data[~high_vol_mask.iloc[i-30:i]]

        if len(high_vol_data) > 5:
            df_enhanced.loc[df_enhanced.index[i], 'eth_btc_corr_highvol'] = (
                high_vol_data['eth_return'].corr(high_vol_data['btc_return'])
            )
        if len(low_vol_data) > 5:
            df_enhanced.loc[df_enhanced.index[i], 'eth_btc_corr_lowvol'] = (
                low_vol_data['eth_return'].corr(low_vol_data['btc_return'])
            )

    return df_enhanced


def remove_raw_prices_and_transform(df):
    df_transformed = df.copy()

    if 'eth_log_return' not in df_transformed.columns:
        df_transformed['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    if 'eth_intraday_range' not in df_transformed.columns:
        df_transformed['eth_intraday_range'] = (df['ETH_High'] - df['ETH_Low']) / (df['ETH_Close'] + 1e-8)
    if 'eth_body_ratio' not in df_transformed.columns:
        df_transformed['eth_body_ratio'] = (df['ETH_Close'] - df['ETH_Open']) / (df['ETH_Close'] + 1e-8)
    if 'eth_close_position' not in df_transformed.columns:
        df_transformed['eth_close_position'] = (
            (df['ETH_Close'] - df['ETH_Low']) / (df['ETH_High'] - df['ETH_Low'] + 1e-8)
        )

    if 'BTC_Close' in df_transformed.columns:
        for period in [5, 20]:
            col_name = f'btc_return_{period}d'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = np.log(df['BTC_Close'] / df['BTC_Close'].shift(period)).fillna(0)
        
        for period in [7, 14, 30]:
            col_name = f'btc_volatility_{period}d'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = (
                    df_transformed['eth_log_return'].rolling(period, min_periods=max(3, period//3)).std()
                ).fillna(0)
        
        if 'btc_intraday_range' not in df_transformed.columns:
            df_transformed['btc_intraday_range'] = (df['BTC_High'] - df['BTC_Low']) / (df['BTC_Close'] + 1e-8)
        if 'btc_body_ratio' not in df_transformed.columns:
            df_transformed['btc_body_ratio'] = (df['BTC_Close'] - df['BTC_Open']) / (df['BTC_Close'] + 1e-8)

        if 'BTC_Volume' in df.columns:
            btc_volume = df['BTC_Volume']
            if 'btc_volume_change' not in df_transformed.columns:
                df_transformed['btc_volume_change'] = btc_volume.pct_change().fillna(0)
            if 'btc_volume_ratio_20d' not in df_transformed.columns:
                volume_ma20 = btc_volume.rolling(20, min_periods=5).mean()
                df_transformed['btc_volume_ratio_20d'] = (btc_volume / (volume_ma20 + 1e-8)).fillna(1)
            if 'btc_volume_volatility_30d' not in df_transformed.columns:
                df_transformed['btc_volume_volatility_30d'] = (
                    btc_volume.pct_change().rolling(30, min_periods=10).std()
                ).fillna(0)
            if 'btc_obv' not in df_transformed.columns:
                btc_close = df['BTC_Close']
                obv = np.where(btc_close > btc_close.shift(1), btc_volume,
                               np.where(btc_close < btc_close.shift(1), -btc_volume, 0))
                df_transformed['btc_obv'] = pd.Series(obv, index=df.index).cumsum().fillna(0)
            if 'btc_volume_price_corr_30d' not in df_transformed.columns:
                df_transformed['btc_volume_price_corr_30d'] = (
                    btc_volume.pct_change().rolling(30, min_periods=10).corr(
                        df_transformed['eth_log_return']
                    )
                ).fillna(0)

    altcoins = ['BNB', 'XRP', 'SOL', 'ADA', 'DOGE', 'AVAX', 'DOT']
    for coin in altcoins:
        if f'{coin}_Close' in df_transformed.columns:
            col_name = f'{coin.lower()}_return'
            if col_name not in df_transformed.columns:
                df_transformed[col_name] = np.log(df[f'{coin}_Close'] / df[f'{coin}_Close'].shift(1)).fillna(0)
            vol_col = f'{coin.lower()}_volatility_30d'
            if vol_col not in df_transformed.columns:
                df_transformed[vol_col] = df_transformed[col_name].rolling(30, min_periods=10).std().fillna(0)
            
            if f'{coin}_Volume' in df.columns:
                coin_volume = df[f'{coin}_Volume']
                volume_change_col = f'{coin.lower()}_volume_change'
                if volume_change_col not in df_transformed.columns:
                    df_transformed[volume_change_col] = coin_volume.pct_change().fillna(0)
                volume_ratio_col = f'{coin.lower()}_volume_ratio_20d'
                if volume_ratio_col not in df_transformed.columns:
                    volume_ma20 = coin_volume.rolling(20, min_periods=5).mean()
                    df_transformed[volume_ratio_col] = (coin_volume / (volume_ma20 + 1e-8)).fillna(1)

    if 'ETH_Volume' in df.columns and 'BTC_Volume' in df.columns:
        eth_volume = df['ETH_Volume']
        btc_volume = df['BTC_Volume']
        if 'eth_btc_volume_corr_30d' not in df_transformed.columns:
            df_transformed['eth_btc_volume_corr_30d'] = (
                eth_volume.pct_change().rolling(30, min_periods=10).corr(btc_volume.pct_change())
            ).fillna(0)
        if 'eth_btc_volume_ratio' not in df_transformed.columns:
            df_transformed['eth_btc_volume_ratio'] = (eth_volume / (btc_volume + 1e-8)).fillna(0)
        if 'eth_btc_volume_ratio_ma30' not in df_transformed.columns:
            df_transformed['eth_btc_volume_ratio_ma30'] = (
                df_transformed['eth_btc_volume_ratio'].rolling(30, min_periods=10).mean()
            ).fillna(0)

    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    cols_to_remove = [
        col for col in df_transformed.columns
        if any(p in col for p in remove_patterns)
        and not any(d in col.lower() for d in ['_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv'])
    ]
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)

    return_cols = [col for col in df_transformed.columns if 'return' in col.lower() and 'next' not in col]
    if return_cols:
        df_transformed[return_cols] = df_transformed[return_cols].fillna(0)

    return df_transformed

In [3]:
def apply_lag_features(df, news_lag=2, onchain_lag=1):
    df_lagged = df.copy()
    
    raw_sentiment_cols = ['sentiment_mean', 'sentiment_std', 'news_count', 'positive_ratio', 'negative_ratio']
    sentiment_ma_cols = [col for col in df.columns if 'sentiment' in col and ('_ma7' in col or '_volatility_7' in col)]
    no_lag_patterns = ['_trend', '_acceleration', '_volume_change', 'news_volume_change', 'news_volume_ma']
    onchain_cols = [col for col in df.columns if any(keyword in col.lower() 
                    for keyword in ['eth_tx', 'eth_active', 'eth_new', 'eth_large', 'eth_token', 
                                  'eth_contract', 'eth_avg_gas', 'eth_total_gas', 'eth_avg_block'])]
    other_cols = [col for col in df.columns if any(keyword in col.lower() 
                  for keyword in ['tvl', 'funding', 'lido_', 'aave_', 'makerdao_', 
                                'chain_', 'usdt_', 'sp500_', 'vix_', 'gold_', 'dxy_', 'fg_'])]
    
    exclude_cols = ['ETH_Close', 'ETH_High', 'ETH_Low', 'ETH_Open', 'date']
    exclude_cols.extend([col for col in df.columns if 'event_' in col or 'period_' in col or '_lag' in col])
    
    cols_to_drop = []
    
    for col in raw_sentiment_cols:
        if col in df.columns:
            for lag in range(1, news_lag + 1):
                df_lagged[f"{col}_lag{lag}"] = df[col].shift(lag)
            cols_to_drop.append(col)
    
    for col in sentiment_ma_cols:
        if col in df.columns and col not in cols_to_drop:
            if not any(pattern in col for pattern in no_lag_patterns):
                df_lagged[f"{col}_lag1"] = df[col].shift(1)
                cols_to_drop.append(col)
    
    for col in onchain_cols:
        if col not in exclude_cols:
            df_lagged[f"{col}_lag1"] = df[col].shift(onchain_lag)
            if col in df.columns:
                cols_to_drop.append(col)
    
    for col in other_cols:
        if col not in exclude_cols:
            df_lagged[f"{col}_lag1"] = df[col].shift(1)
            if col in df.columns:
                cols_to_drop.append(col)
    
    df_lagged.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    return df_lagged


def add_price_lag_features_first(df):
    df_new = df.copy()
    close = df['ETH_Close']
    high = df['ETH_High']
    low = df['ETH_Low']
    volume = df['ETH_Volume']
    
    for lag in [1, 2, 3, 5, 7, 14, 21, 30]:
        df_new[f'close_lag{lag}'] = close.shift(lag)
    
    for lag in [1, 2, 3, 5, 7]:
        df_new[f'high_lag{lag}'] = high.shift(lag)
        df_new[f'low_lag{lag}'] = low.shift(lag)
        df_new[f'volume_lag{lag}'] = volume.shift(lag)
        df_new[f'return_lag{lag}'] = close.pct_change(periods=lag).shift(1)
    
    for lag in [1, 7, 30]:
        df_new[f'close_ratio_lag{lag}'] = close / close.shift(lag)
    
    return df_new


def create_targets(df):
    df_target = df.copy()
    next_open = df['ETH_Open'].shift(-1)
    next_close = df['ETH_Close'].shift(-1)
    df_target['next_log_return'] = np.log(next_close / next_open)
    df_target['next_direction'] = (next_close > next_open).astype(int)
    df_target['next_open'] = next_open
    df_target['next_close'] = next_close
    return df_target


In [4]:
def add_interaction_features(df):
    df_interact = df.copy()
    
    if 'RSI_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['RSI_Volume_Strength'] = df['RSI_14'] * df['VOLUME_RATIO']
    
    if 'vix_VIX' in df.columns and 'VOLATILITY_20' in df.columns:
        df_interact['VIX_ETH_Vol_Cross'] = df['vix_VIX'] * df['VOLATILITY_20']
    
    if 'MACD_12_26_9' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['MACD_Volume_Momentum'] = df['MACD_12_26_9'] * df['VOLUME_RATIO']
    
    if 'btc_return' in df.columns and 'eth_btc_corr_30d' in df.columns:
        df_interact['BTC_Weighted_Impact'] = df['btc_return'] * df['eth_btc_corr_30d']
    
    if 'ATR_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['Liquidity_Risk'] = df['ATR_14'] * (1 / (df['VOLUME_RATIO'] + 1e-8))
    
    return df_interact

def add_volatility_regime_features(df):
    df_regime = df.copy()
    
    if 'VOLATILITY_20' in df.columns:
        vol_median = df['VOLATILITY_20'].rolling(60, min_periods=20).median()
        df_regime['vol_regime_high'] = (df['VOLATILITY_20'] > vol_median).astype(int)
        
        vol_mean = df['VOLATILITY_20'].rolling(30, min_periods=10).mean()
        vol_std = df['VOLATILITY_20'].rolling(30, min_periods=10).std()
        df_regime['vol_spike'] = (df['VOLATILITY_20'] > vol_mean + 2 * vol_std).astype(int)
        
        df_regime['vol_percentile_90d'] = df['VOLATILITY_20'].rolling(90, min_periods=30).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
        df_regime['vol_trend'] = df['VOLATILITY_20'].pct_change(5)
        df_regime['vol_regime_duration'] = df_regime.groupby(
            (df_regime['vol_regime_high'] != df_regime['vol_regime_high'].shift()).cumsum()
        ).cumcount() + 1

    return df_regime


def add_normalized_price_lags(df):
    df_norm = df.copy()
    
    if 'ETH_Close' not in df.columns:
        return df_norm
    
    current_close = df['ETH_Close']
    lag_cols = [col for col in df.columns if 'close_lag' in col and col.replace('close_lag', '').isdigit()]
    
    for col in lag_cols:
        lag_num = col.replace('close_lag', '')
        df_norm[f'close_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
        next_lag_col = f'close_lag{int(lag_num)+1}'
        if next_lag_col in df.columns:
            df_norm[f'close_lag{lag_num}_logret'] = np.log(df[col] / (df[next_lag_col] + 1e-8))
    
    for col in df.columns:
        if 'high_lag' in col:
            lag_num = col.replace('high_lag', '')
            df_norm[f'high_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
        if 'low_lag' in col:
            lag_num = col.replace('low_lag', '')
            df_norm[f'low_lag{lag_num}_ratio'] = df[col] / (current_close + 1e-8)
    
    return df_norm


def add_percentile_features(df):
    df_pct = df.copy()
    
    if 'ETH_Close' in df.columns:
        df_pct['price_percentile_250d'] = df['ETH_Close'].rolling(250, min_periods=60).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    if 'ETH_Volume' in df.columns:
        df_pct['volume_percentile_90d'] = df['ETH_Volume'].rolling(90, min_periods=30).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    if 'RSI_14' in df.columns:
        df_pct['RSI_percentile_60d'] = df['RSI_14'].rolling(60, min_periods=20).apply(
            lambda x: (x.iloc[-1] > x).sum() / len(x) if len(x) > 0 else 0.5
        )
    
    return df_pct


def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """
    암호화폐 시계열 결측치 처리
    
    참고문헌:
    1. "Quantifying Cryptocurrency Unpredictability" (2025)

    2. "Time Series Data Forecasting" 
    
    3. "Dealing with Leaky Missing Data in Production" (2021)

    """
    
    # ===== 1. Lookback 제거 =====
    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    
    before = len(df_clean)
    df_clean = df_clean[df_clean['date'] >= train_start_date].reset_index(drop=True)
    
    # ===== 2. Feature 컬럼 선택 =====
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open']
    feature_cols = [col for col in df_clean.columns 
                   if col not in target_cols + ['date']]
    
    # ===== 3. 결측 확인 =====
    missing_before = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 4. FFill → 0 =====
    df_clean[feature_cols] = df_clean[feature_cols].fillna(method='ffill')
    df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    missing_after = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 5. 무한대 처리 =====
    inf_count = 0
    for col in feature_cols:
        if np.isinf(df_clean[col]).sum() > 0:
            inf_count += np.isinf(df_clean[col]).sum()
            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
            df_clean[col] = df_clean[col].fillna(method='ffill').fillna(0)
    
    # ===== 6. 최종 확인 =====
    final_missing = df_clean[feature_cols].isnull().sum().sum()
    
    if final_missing > 0:
        df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    
    if is_train:
        return df_clean, {}
    else:
        return df_clean

In [5]:
def select_features_multi_target(X_train, y_train, target_type='direction', top_n=30):
    
    if target_type == 'direction':
        selected, stats = select_features_verified(
            X_train, 
            y_train['next_direction'], 
            task='class', 
            top_n=top_n
        )
        
    elif target_type == 'return':
        selected, stats = select_features_verified(
            X_train, 
            y_train['next_log_return'], 
            task='reg', 
            top_n=top_n
        )
        
    elif target_type == 'price':
        selected, stats = select_features_verified(
            X_train, 
            y_train['next_close'], 
            task='reg', 
            top_n=top_n
        )
        
    elif target_type == 'direction_return':
        print("\n[Hybrid] Direction (50%) + Return (50%)")
        
        dir_features, dir_stats = select_features_verified(
            X_train, 
            y_train['next_direction'], 
            task='class', 
            top_n=top_n // 2,
            verbose=False
        )
        
        ret_features, ret_stats = select_features_verified(
            X_train, 
            y_train['next_log_return'], 
            task='reg', 
            top_n=top_n // 2,
            verbose=False
        )
        
        selected = list(dict.fromkeys(dir_features + ret_features))
        
        if len(selected) < top_n:
            all_mi_scores = {**dir_stats['mi_scores'], **ret_stats['mi_scores']}
            sorted_features = sorted(all_mi_scores.items(), key=lambda x: x[1], reverse=True)
            
            for feat, _ in sorted_features:
                if feat not in selected:
                    selected.append(feat)
                    if len(selected) >= top_n:
                        break
        
        selected = selected[:top_n]
        
        stats = {
            'dir_stats': dir_stats,
            'ret_stats': ret_stats,
            'overlap': len(set(dir_features) & set(ret_features))
        }
        
        
    elif target_type == 'direction_price':
        print("\n[Hybrid] Direction (50%) + Price (50%)")
        
        dir_features, dir_stats = select_features_verified(
            X_train, 
            y_train['next_direction'], 
            task='class', 
            top_n=top_n // 2,
            verbose=False
        )
        
        price_features, price_stats = select_features_verified(
            X_train, 
            y_train['next_close'], 
            task='reg', 
            top_n=top_n // 2,
            verbose=False
        )
        
        selected = list(dict.fromkeys(dir_features + price_features))
        
        if len(selected) < top_n:
            all_mi_scores = {**dir_stats['mi_scores'], **price_stats['mi_scores']}
            sorted_features = sorted(all_mi_scores.items(), key=lambda x: x[1], reverse=True)
            
            for feat, _ in sorted_features:
                if feat not in selected:
                    selected.append(feat)
                    if len(selected) >= top_n:
                        break
        
        selected = selected[:top_n]
        
        stats = {
            'dir_stats': dir_stats,
            'price_stats': price_stats,
            'overlap': len(set(dir_features) & set(price_features))
        }
        
    else:
        raise ValueError(f"Unknown target_type: {target_type}")
    
    print("Selected Features")
    print(", ".join(selected))
    return selected, stats


def select_features_verified(X_train, y_train, task='class', top_n=30, verbose=True):
    
    if task == 'class':
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
    
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    if task == 'class':
        estimator = LGBMClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    else:
        estimator = LGBMRegressor(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    
    rfe = RFE(
        estimator=estimator,
        n_features_to_select=top_n,
        step=0.1,
        verbose=0
    )
    
    rfe.fit(X_train, y_train)
    rfe_features = X_train.columns[rfe.support_].tolist()

    if task == 'class':
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    else:
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    
    rf_model.fit(X_train, y_train)
    rf_importances = rf_model.feature_importances_
    rf_idx = np.argsort(rf_importances)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]

    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0:
                    break
    
    return selected_features, {
        'mi_features': mi_features,
        'rfe_features': rfe_features,
        'rf_features': rf_features,
        'feature_votes': feature_votes,
        'mi_scores': dict(zip(X_train.columns, mi_scores)),
        'rf_importances': dict(zip(X_train.columns, rf_importances))
    }


def split_tvt_method(df, train_start_date, test_start_date='2025-01-01', 
                     train_ratio=0.7, val_ratio=0.15):
    """
    test_start_date를 고정하고, 그 이전 데이터를 train/val로 분할
    test_start_date 이후 데이터는 모두 test로 사용
    """
    df_period = df[df['date'] >= train_start_date].copy()
    
    # 테스트 시작 날짜를 datetime으로 변환
    if isinstance(test_start_date, str):
        test_start_date = pd.to_datetime(test_start_date)
    
    # test_start_date 이전 데이터를 train/val로, 이후를 test로 분할
    pre_test_df = df_period[df_period['date'] < test_start_date].copy()
    test_df = df_period[df_period['date'] >= test_start_date].copy()
    
    # train/val 분할 (test 이전 데이터만 사용)
    n_pre_test = len(pre_test_df)
    train_end = int(n_pre_test * train_ratio / (train_ratio + val_ratio))
    
    train_df = pre_test_df.iloc[:train_end].copy()
    val_df = pre_test_df.iloc[train_end:].copy()
    
    print(f"\n{'='*80}")
    print(f"TVT Split (Fixed Test Start: {test_start_date.date()})")
    print(f"{'='*80}")
    print(f"  Train: {len(train_df):4d} ({train_df['date'].min().date()} ~ {train_df['date'].max().date()})")
    print(f"  Val:   {len(val_df):4d} ({val_df['date'].min().date()} ~ {val_df['date'].max().date()})")
    print(f"  Test:  {len(test_df):4d} ({test_df['date'].min().date()} ~ {test_df['date'].max().date()})")
    print(f"{'='*80}\n")
    
    return {'train': train_df, 'val': val_df, 'test': test_df}

def split_walk_forward_method(df, train_start_date, 
                              final_test_start='2025-01-01',
                              initial_train_size=800,    
                              val_size=150,             
                              test_size=150,           
                              step=150,                  
                              gap_size=7):
    """
    Reverse Rolling Walk-Forward Validation
    - 마지막 날짜부터 시작해서 과거로 rolling
    - Train 크기는 고정 (initial_train_size)
    - Final holdout은 2025-01-01부터 고정
    """
    
    df_period = df[df['date'] >= train_start_date].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    
    total_days = len(df_period)
    min_required_days = initial_train_size + val_size + (gap_size * 2) + test_size
    n_splits = (total_days - min_required_days) // step + 1
    
    print(f"\n{'='*80}")
    print(f"Reverse Rolling Walk-Forward Configuration ")
    print(f"{'='*80}")
    print(f"Total: {len(df_period)} days")
    print(f"Rolling train size: {initial_train_size} days (FIXED)")
    print(f"Val: {val_size} days | Test: {test_size} days")
    print(f"Gap: {gap_size} days | Step: {step} days (BACKWARD)")
    print(f"Target: {n_splits} walk-forward + 1 final holdout")
    print(f"{'='*80}\n")
    
    folds = []
    
    # 역방향 rolling
    for fold_idx in range(n_splits):
        test_end_idx = total_days - (fold_idx * step)
        test_start_idx = test_end_idx - test_size
        
        if test_start_idx < 0:
            break
        
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0:
            break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_idx': fold_idx + 1,
            'fold_type': 'walk_forward_rolling_reverse'
        })
    
    # 시간순으로 정렬
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
        print(f"Fold {fold['fold_idx']} (walk_forward_rolling)")
        print(f"  Train: {len(fold['train']):4d}d  {fold['train']['date'].min().date()} ~ {fold['train']['date'].max().date()}")
        print(f"  Val:   {len(fold['val']):4d}d  {fold['val']['date'].min().date()} ~ {fold['val']['date'].max().date()}")
        print(f"  Test:  {len(fold['test']):4d}d  {fold['test']['date'].min().date()} ~ {fold['test']['date'].max().date()}\n")
    
    # Final holdout
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        
        final_val_end_idx = len(pre_final_df)
        final_val_start_idx = final_val_end_idx - val_size
        final_train_end_idx = final_val_start_idx - gap_size
        final_train_start_idx = final_train_end_idx - initial_train_size
        
        if final_train_start_idx < 0:
            final_train_start_idx = 0
        
        final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
        final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
        
        print(f"Fold {len(folds) + 1} (final_holdout)")
        print(f"  Train: {len(final_train_data):4d}d  {final_train_data['date'].min().date()} ~ {final_train_data['date'].max().date()}")
        print(f"  Val:   {len(final_val_data):4d}d  {final_val_data['date'].min().date()} ~ {final_val_data['date'].max().date()}")
        print(f"  Test:  {len(final_test_df):4d}d  {final_test_df['date'].min().date()} ~ {final_test_df['date'].max().date()}\n")
        
        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })
    
    print(f"{'='*80}")
    print(f"Created {len(folds)} folds total")
    print(f"{'='*80}\n")
    
    return folds


def process_single_split(split_data, target_type='direction', top_n=40, fold_idx=None):
    """
    각 fold를 독립적으로 처리 (feature selection 포함)
    """
    
    train_df = split_data['train']
    val_df = split_data['val']
    test_df = split_data['test']
    fold_type = split_data.get('fold_type', 'unknown')
    
    if fold_idx is not None:
        print(f"\n{'='*60}")
        print(f"Processing Fold {fold_idx} ({fold_type})")
        print(f"{'='*60}")
    
    train_processed, missing_stats = handle_missing_values_paper_based(
        train_df.copy(),
        train_start_date=train_df['date'].min(),
        is_train=True
    )
    
    val_processed = handle_missing_values_paper_based(
        val_df.copy(),
        train_start_date=val_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    test_processed = handle_missing_values_paper_based(
        test_df.copy(),
        train_start_date=test_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open']
    
    train_processed = train_processed.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_processed.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_processed.dropna(subset=target_cols).reset_index(drop=True)

    feature_cols = [col for col in train_processed.columns 
                   if col not in target_cols + ['date']]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    print(f"\n[Feature Selection for Fold {fold_idx}]")
    print(f"Training data shape: {X_train.shape}")
    
    selected_features, selection_stats = select_features_multi_target(
        X_train, 
        y_train, 
        target_type=target_type, 
        top_n=top_n
    )
    
    print(f"Selected {len(selected_features)} features for this fold")
    
    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]
    
    robust_scaler = RobustScaler()
    standard_scaler = StandardScaler()
    
    X_train_robust = robust_scaler.fit_transform(X_train_sel)
    X_val_robust = robust_scaler.transform(X_val_sel)
    X_test_robust = robust_scaler.transform(X_test_sel)
    
    X_train_standard = standard_scaler.fit_transform(X_train_sel)
    X_val_standard = standard_scaler.transform(X_val_sel)
    X_test_standard = standard_scaler.transform(X_test_sel)
    
    print(f"Scaling completed for Fold {fold_idx}")
    print(f"{'='*60}\n")
    
    result = {
        'train': {
            'X_robust': X_train_robust,
            'X_standard': X_train_standard,
            'X_raw': X_train_sel,
            'y': y_train.reset_index(drop=True), 
            'dates': train_df['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_robust,
            'X_standard': X_val_standard,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_df['date'].reset_index(drop=True)  
        },
        'test': {
            'X_robust': X_test_robust,
            'X_standard': X_test_standard,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),  
            'dates': test_df['date'].reset_index(drop=True)  
        },
        'scaler': robust_scaler, 
        'stats': {
            'robust_scaler': robust_scaler,
            'standard_scaler': standard_scaler,
            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx
        }
    }
    
    return result



def build_complete_pipeline_corrected(df_raw, train_start_date, 
                                     final_test_start='2025-01-01',
                                     method='tvt', target_type='direction', **kwargs):
    """
    전체 파이프라인 실행 함수
    
    Parameters:
    -----------
    df_raw : DataFrame
        원본 데이터
    train_start_date : str
        학습 데이터 시작 날짜
    final_test_start : str, default='2025-01-01'
        최종 고정 테스트 시작 날짜
        - TVT: 이 날짜부터 마지막까지 테스트
        - Walk-forward: 이 날짜 이전은 walk-forward folds, 이후는 final holdout
    method : str, default='tvt'
        'tvt' 또는 'walk_forward'
    target_type : str, default='direction'
        'direction', 'return', 'price', 'direction_return', 'direction_price'
    **kwargs : dict
        각 method에 필요한 추가 파라미터
    """
    
    df = df_raw.copy()

    df = create_targets(df)
    df = add_price_lag_features_first(df)
    df = calculate_technical_indicators(df)
    df = add_enhanced_cross_crypto_features(df)
    df = add_volatility_regime_features(df)
    df = add_interaction_features(df)
    df = add_percentile_features(df)
    df = add_normalized_price_lags(df)
    df = remove_raw_prices_and_transform(df)
    df = apply_lag_features(df, news_lag=2, onchain_lag=1)


    pd.set_option('display.max_columns', None)
    df = df.iloc[:-1]  
    
    split_kwargs = {}
    
    if method == 'tvt':
        split_kwargs['test_start_date'] = final_test_start
        if 'train_ratio' in kwargs:
            split_kwargs['train_ratio'] = kwargs['train_ratio']
        if 'val_ratio' in kwargs:
            split_kwargs['val_ratio'] = kwargs['val_ratio']
        splits = split_tvt_method(df, train_start_date, **split_kwargs)
        
    elif method == 'walk_forward':
        split_kwargs['final_test_start'] = final_test_start
        if 'n_splits' in kwargs:
            split_kwargs['n_splits'] = kwargs['n_splits']
        if 'initial_train_size' in kwargs:
            split_kwargs['initial_train_size'] = kwargs['initial_train_size']
        if 'test_size' in kwargs:
            split_kwargs['test_size'] = kwargs['test_size']
        if 'val_size' in kwargs:
            split_kwargs['val_size'] = kwargs['val_size']
        if 'step' in kwargs:
            split_kwargs['step'] = kwargs['step']
        splits = split_walk_forward_method(df, train_start_date, **split_kwargs)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    if method == 'tvt':
        result = process_single_split(
            splits, 
            target_type=target_type,  
            top_n=30,
            fold_idx=1
        )
    else:
        result = [
            process_single_split(
                fold, 
                target_type=target_type,  
                top_n=30,
                fold_idx=fold['fold_idx']
            ) 
            for fold in splits
        ]
    
    return result


## dropout 설정때매 오류난다 캐서 그거 변경한 버전

In [6]:



class TimeSeriesAugmentation:
    """
    시계열 데이터 증강을 위한 유틸리티 클래스
    """
    
    @staticmethod
    def jittering(X, sigma=0.02):
        """
        가우시안 노이즈 추가
        """
        noise = np.random.normal(0, sigma, X.shape)
        return X + noise
    
    @staticmethod
    def scaling(X, sigma=0.1):
        """
        랜덤 스케일링 적용
        """
        if len(X.shape) == 3:
            factor = np.random.normal(1, sigma, (X.shape[0], 1, X.shape[2]))
        else:
            factor = np.random.normal(1, sigma, (X.shape[0], X.shape[1]))
        return X * factor
    
    @staticmethod
    def magnitude_warping(X, sigma=0.2, num_knots=4):
        """
        진폭 왜곡 적용
        """
        if len(X.shape) == 3:
            seq_len = X.shape[1]
            orig_steps = np.linspace(0, seq_len - 1, num_knots + 2)
            random_warps = np.random.normal(1, sigma, size=(X.shape[0], num_knots + 2, X.shape[2]))
            
            warped_X = np.zeros_like(X)
            for i in range(X.shape[0]):
                for j in range(X.shape[2]):
                    warper = np.interp(np.arange(seq_len), orig_steps, random_warps[i, :, j])
                    warped_X[i, :, j] = X[i, :, j] * warper
            return warped_X
        else:
            return X * np.random.normal(1, sigma, X.shape)
    
    @staticmethod
    def apply_augmentation(X, method='jittering', **kwargs):
        """
        선택된 증강 기법 적용
        """
        if method == 'jittering':
            return TimeSeriesAugmentation.jittering(X, **kwargs)
        elif method == 'scaling':
            return TimeSeriesAugmentation.scaling(X, **kwargs)
        elif method == 'magnitude_warping':
            return TimeSeriesAugmentation.magnitude_warping(X, **kwargs)
        else:
            return X


class DirectionModels:
    
    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 80, 200),
                'max_depth': trial.suggest_int('max_depth', 4, 8),
                'min_samples_split': trial.suggest_int('min_samples_split', 40, 70),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 35),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'max_samples': trial.suggest_float('max_samples', 0.6, 0.8),
                'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 40, 100),
                'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.01),
                'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.01),
                'random_state': 42,
                'n_jobs': -1,
                'bootstrap': True
            }
            
            model = RandomForestClassifier(**param)
            model.fit(X_train, y_train)
            
            train_acc = model.score(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0)
        )
        
        study.optimize(objective, n_trials=30, show_progress_bar=False, n_jobs=1)
        
        best_model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1, bootstrap=True)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Random Forest] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 150, 400),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 15, 50),
                'subsample': trial.suggest_float('subsample', 0.5, 0.8),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 20.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 20.0, log=True),
                'min_child_samples': trial.suggest_int('min_child_samples', 50, 100),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0, log=True),
                'min_split_gain': trial.suggest_float('min_split_gain', 0.01, 1.0, log=True),
                'path_smooth': trial.suggest_float('path_smooth', 0.0, 1.0),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.8),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.8),
                'bagging_freq': 1,
                'random_state': 42,
                'verbose': -1,
                'force_col_wise': True
            }

            model = LGBMClassifier(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='binary_logloss',
                callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
            )

            train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
            train_acc = accuracy_score(y_train, train_pred)
            val_acc = accuracy_score(y_val, y_val_pred)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty

        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=8),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=10)
        )

        study.optimize(objective, n_trials=30, show_progress_bar=False)

        best_params = study.best_params
        best_params['random_state'] = 42
        best_params['verbose'] = -1
        best_params['force_col_wise'] = True
        best_params['bagging_freq'] = 1

        final_model = LGBMClassifier(**best_params)
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
        )

        train_pred = final_model.predict(X_train)
        val_pred = final_model.predict(X_val)
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"[LightGBM] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")

        return final_model
    
    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 150, 400),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 0.8),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 0.8),
                'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 0.8),
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 20.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 2.0, 20.0, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 10, 30),
                'gamma': trial.suggest_float('gamma', 0.1, 2.0, log=True),
                'max_delta_step': trial.suggest_float('max_delta_step', 0, 3),
                'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.8, 1.5),
                'random_state': 42,
                'n_jobs': -1,
                'tree_method': 'hist',
                'eval_metric': 'logloss'
            }

            model = XGBClassifier(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                verbose=False
            )

            train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
            train_acc = accuracy_score(y_train, train_pred)
            val_acc = accuracy_score(y_val, y_val_pred)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty

        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=8),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=10)
        )

        study.optimize(objective, n_trials=30, show_progress_bar=False)

        best_params = study.best_params
        best_params['random_state'] = 42
        best_params['n_jobs'] = -1
        best_params['tree_method'] = 'hist'
        best_params['eval_metric'] = 'logloss'

        final_model = XGBClassifier(**best_params)
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        train_pred = final_model.predict(X_train)
        val_pred = final_model.predict(X_val)
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"[XGBoost] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")

        return final_model

    @staticmethod
    def histgradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'max_iter': trial.suggest_int('max_iter', 100, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 6),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 25, 70),
                'l2_regularization': trial.suggest_float('l2_regularization', 1.0, 20.0, log=True),
                'max_bins': trial.suggest_int('max_bins', 128, 255),
                'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 15, 40),
                'early_stopping': True,
                'n_iter_no_change': 20,
                'validation_fraction': 0.1,
                'random_state': 42
            }
            
            model = HistGradientBoostingClassifier(**params)
            model.fit(X_train, y_train)
            
            train_acc = model.score(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10)
        )
        
        study.optimize(objective, n_trials=30, show_progress_bar=False)
        
        best_model = HistGradientBoostingClassifier(**study.best_params)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[HistGradientBoosting] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model

    @staticmethod
    def svm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'C': trial.suggest_float('C', 0.01, 1.0, log=True),
                'gamma': trial.suggest_float('gamma', 0.0001, 0.01, log=True),
                'kernel': 'rbf',
                'probability': True,
                'random_state': 42,
                'cache_size': 2000,
                'max_iter': 2000
            }
            
            model = SVC(**param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=8),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=4)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        best_model = SVC(**study.best_params, random_state=42, probability=True, cache_size=2000)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[SVM] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model

    @staticmethod
    def logistic_regression(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'C': trial.suggest_float('C', 0.01, 5.0, log=True),
                'penalty': 'l2',
                'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga']),
                'max_iter': 3000,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = LogisticRegression(**param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_model = LogisticRegression(**study.best_params)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Logistic Regression] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def naive_bayes(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            var_smoothing = trial.suggest_float('var_smoothing', 1e-11, 1e-5, log=True)
            
            model = GaussianNB(var_smoothing=var_smoothing)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42)
        )
        
        study.optimize(objective, n_trials=15, show_progress_bar=False)
        
        model = GaussianNB(var_smoothing=study.best_params['var_smoothing'])
        model.fit(X_train, y_train)
        
        train_acc = model.score(X_train, y_train)
        val_acc = model.score(X_val, y_val)
        print(f"[Naive Bayes] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return model
    
    @staticmethod
    def knn(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'n_neighbors': trial.suggest_int('n_neighbors', 15, 35),
                'weights': 'distance',
                'metric': 'manhattan',
                'leaf_size': trial.suggest_int('leaf_size', 30, 80),
                'p': 1,
                'n_jobs': -1
            }
            
            model = KNeighborsClassifier(**param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_model = KNeighborsClassifier(**study.best_params)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[KNN] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def adaboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 30, 100),
                'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.5),
                'algorithm': 'SAMME',
                'random_state': 42
            }
            
            base_max_depth = trial.suggest_int('base_max_depth', 1, 3)
            base_min_samples_split = trial.suggest_int('base_min_samples_split', 30, 60)
            base_min_samples_leaf = trial.suggest_int('base_min_samples_leaf', 15, 30)
            
            base_estimator = DecisionTreeClassifier(
                max_depth=base_max_depth,
                min_samples_split=base_min_samples_split,
                min_samples_leaf=base_min_samples_leaf,
                max_features='sqrt',
                random_state=42
            )
            
            model = AdaBoostClassifier(estimator=base_estimator, **param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        best_params = study.best_params
        base_estimator = DecisionTreeClassifier(
            max_depth=best_params['base_max_depth'],
            min_samples_split=best_params['base_min_samples_split'],
            min_samples_leaf=best_params['base_min_samples_leaf'],
            max_features='sqrt',
            random_state=42
        )
        
        best_model = AdaBoostClassifier(
            estimator=base_estimator,
            n_estimators=best_params['n_estimators'],
            learning_rate=best_params['learning_rate'],
            algorithm='SAMME',
            random_state=42
        )
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[AdaBoost] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'iterations': trial.suggest_int('iterations', 100, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'depth': trial.suggest_int('depth', 2, 4),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 8.0, 20.0),
                'subsample': trial.suggest_float('subsample', 0.4, 0.7),
                'rsm': trial.suggest_float('rsm', 0.4, 0.7),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 40, 80),
                'random_seed': 42,
                'verbose': False,
                'early_stopping_rounds': 20
            }
            
            model = CatBoostClassifier(**param)
            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                verbose=False
            )
            
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=15)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        model = CatBoostClassifier(**study.best_params, random_seed=42, verbose=False)
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            verbose=False
        )
        
        train_acc = model.score(X_train, y_train)
        val_acc = model.score(X_val, y_val)
        print(f"[CatBoost] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return model
    
    @staticmethod
    def decision_tree(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'max_depth': trial.suggest_int('max_depth', 5, 10),
                'min_samples_split': trial.suggest_int('min_samples_split', 30, 70),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 15, 35),
                'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
                'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 40, 120),
                'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.02),
                'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.02),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42
            }
            
            model = DecisionTreeClassifier(**param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10)
        )
        
        study.optimize(objective, n_trials=30, show_progress_bar=False)
        
        best_model = DecisionTreeClassifier(**study.best_params)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Decision Tree] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def extra_trees(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            n_estimators = trial.suggest_int('n_estimators', 50, 150, step=25)
            max_depth = trial.suggest_int('max_depth', 5, 15)
            min_samples_split = trial.suggest_int('min_samples_split', 20, 60)
            min_samples_leaf = trial.suggest_int('min_samples_leaf', 10, 30)
            max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
            max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 30, 100)
            min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0, 0.02)
            ccp_alpha = trial.suggest_float('ccp_alpha', 0.0, 0.02)
            bootstrap = trial.suggest_categorical('bootstrap', [True, False])

            if bootstrap:
                max_samples = trial.suggest_float('max_samples', 0.5, 0.8)
            else:
                max_samples = None

            model = ExtraTreesClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                max_features=max_features,
                max_leaf_nodes=max_leaf_nodes,
                min_impurity_decrease=min_impurity_decrease,
                ccp_alpha=ccp_alpha,
                bootstrap=bootstrap,
                max_samples=max_samples,
                random_state=42,
                n_jobs=-1
            )

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            return accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        best_params = study.best_params
        bootstrap = best_params.pop('bootstrap')
        max_samples = best_params.pop('max_samples', None)

        model = ExtraTreesClassifier(
            **best_params,
            bootstrap=bootstrap,
            max_samples=max_samples if bootstrap else None,
            random_state=42,
            n_jobs=-1
        )

        model.fit(X_train, y_train)

        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"[ExtraTrees] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")

        return model
    
    @staticmethod
    def bagging(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            base_max_depth = trial.suggest_int('base_max_depth', 6, 12)
            base_min_samples_split = trial.suggest_int('base_min_samples_split', 25, 50)
            base_min_samples_leaf = trial.suggest_int('base_min_samples_leaf', 10, 25)
            
            base_estimator = DecisionTreeClassifier(
                max_depth=base_max_depth,
                min_samples_split=base_min_samples_split,
                min_samples_leaf=base_min_samples_leaf,
                random_state=42
            )
            
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 80, 200),
                'max_samples': trial.suggest_float('max_samples', 0.6, 0.8),
                'max_features': trial.suggest_float('max_features', 0.6, 0.8),
                'bootstrap': True,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = BaggingClassifier(estimator=base_estimator, **param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        best_params = study.best_params
        base_estimator = DecisionTreeClassifier(
            max_depth=best_params['base_max_depth'],
            min_samples_split=best_params['base_min_samples_split'],
            min_samples_leaf=best_params['base_min_samples_leaf'],
            random_state=42
        )
        
        best_model = BaggingClassifier(
            estimator=base_estimator,
            n_estimators=best_params['n_estimators'],
            max_samples=best_params['max_samples'],
            max_features=best_params['max_features'],
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        )
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Bagging] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def gradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 80, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 5),
                'subsample': trial.suggest_float('subsample', 0.4, 0.7),
                'min_samples_split': trial.suggest_int('min_samples_split', 40, 80),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 40),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 30, 80),
                'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.02),
                'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.02),
                'validation_fraction': 0.15,
                'n_iter_no_change': 15,
                'tol': 0.001,
                'random_state': 42
            }
            
            model = GradientBoostingClassifier(**param)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=12),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        best_model = GradientBoostingClassifier(**study.best_params)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Gradient Boosting] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def mlp(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            units1 = trial.suggest_int('units1', 64, 160, step=32)
            units2 = trial.suggest_int('units2', 32, 80, step=16)
            units3 = trial.suggest_int('units3', 16, 48, step=8)
            dropout = trial.suggest_float('dropout', 0.4, 0.7)
            l2_reg = trial.suggest_float('l2_reg', 0.01, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)
            
            input_dim = X_train.shape[1]
            model = Sequential([
                Dense(units1, activation='relu', input_dim=input_dim, kernel_regularizer=l2(l2_reg)),
                BatchNormalization(),
                Dropout(dropout),
                Dense(units2, activation='relu', kernel_regularizer=l2(l2_reg)),
                BatchNormalization(),
                Dropout(dropout * 0.8),
                Dense(units3, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout * 0.6),
                Dense(1, activation='sigmoid')
            ])
            
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=1e-4, mode='min')
            
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=60,
                batch_size=64,
                callbacks=[early_stop],
                verbose=0
            )
            
            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=8)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_params = study.best_params
        input_dim = X_train.shape[1]
        model = Sequential([
            Dense(best_params['units1'], activation='relu', input_dim=input_dim, kernel_regularizer=l2(best_params['l2_reg'])),
            BatchNormalization(),
            Dropout(best_params['dropout']),
            Dense(best_params['units2'], activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            BatchNormalization(),
            Dropout(best_params['dropout'] * 0.8),
            Dense(best_params['units3'], activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout'] * 0.6),
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=8, min_lr=1e-7, mode='min', verbose=0)
        
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=120,
            batch_size=64,
            callbacks=[early_stop, reduce_lr],
            verbose=0
        )
        
        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[MLP] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return model
    
    @staticmethod
    def stacking_ensemble(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            xgb_estimators = trial.suggest_int('xgb_estimators', 100, 200)
            xgb_depth = trial.suggest_int('xgb_depth', 3, 5)
            xgb_lr = trial.suggest_float('xgb_lr', 0.01, 0.05, log=True)
            
            lgbm_estimators = trial.suggest_int('lgbm_estimators', 100, 200)
            lgbm_depth = trial.suggest_int('lgbm_depth', 3, 5)
            lgbm_lr = trial.suggest_float('lgbm_lr', 0.01, 0.05, log=True)
            
            meta_C = trial.suggest_float('meta_C', 0.1, 2.0, log=True)
            
            base_learners = [
                ('xgb', XGBClassifier(
                    n_estimators=xgb_estimators,
                    max_depth=xgb_depth,
                    learning_rate=xgb_lr,
                    subsample=0.6,
                    colsample_bytree=0.6,
                    reg_alpha=2.0,
                    reg_lambda=3.0,
                    min_child_weight=10,
                    random_state=42,
                    n_jobs=-1
                )),
                ('lgbm', LGBMClassifier(
                    n_estimators=lgbm_estimators,
                    max_depth=lgbm_depth,
                    learning_rate=lgbm_lr,
                    subsample=0.6,
                    colsample_bytree=0.6,
                    reg_alpha=2.0,
                    reg_lambda=2.0,
                    min_child_samples=60,
                    random_state=42,
                    verbose=-1,
                    force_col_wise=True
                ))
            ]
            
            meta_learner = LogisticRegression(max_iter=3000, C=meta_C, random_state=42, penalty='l2')
            
            model = StackingClassifier(
                estimators=base_learners,
                final_estimator=meta_learner,
                cv=7,
                n_jobs=-1,
                passthrough=False
            )
            
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=3)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_params = study.best_params
        base_learners = [
            ('xgb', XGBClassifier(
                n_estimators=best_params['xgb_estimators'],
                max_depth=best_params['xgb_depth'],
                learning_rate=best_params['xgb_lr'],
                subsample=0.6,
                colsample_bytree=0.6,
                reg_alpha=2.0,
                reg_lambda=3.0,
                min_child_weight=10,
                random_state=42,
                n_jobs=-1
            )),
            ('lgbm', LGBMClassifier(
                n_estimators=best_params['lgbm_estimators'],
                max_depth=best_params['lgbm_depth'],
                learning_rate=best_params['lgbm_lr'],
                subsample=0.6,
                colsample_bytree=0.6,
                reg_alpha=2.0,
                reg_lambda=2.0,
                min_child_samples=60,
                random_state=42,
                verbose=-1,
                force_col_wise=True
            ))
        ]
        
        meta_learner = LogisticRegression(max_iter=3000, C=best_params['meta_C'], random_state=42, penalty='l2')
        
        best_model = StackingClassifier(
            estimators=base_learners,
            final_estimator=meta_learner,
            cv=7,
            n_jobs=-1,
            passthrough=False
        )
        
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Stacking] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def voting_hard(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            xgb_n_est = trial.suggest_int('xgb_n_estimators', 100, 200)
            xgb_depth = trial.suggest_int('xgb_max_depth', 3, 5)
            lgbm_n_est = trial.suggest_int('lgbm_n_estimators', 100, 200)
            
            estimators = [
                ('xgb', XGBClassifier(
                    n_estimators=xgb_n_est,
                    max_depth=xgb_depth,
                    learning_rate=0.03,
                    subsample=0.6,
                    colsample_bytree=0.6,
                    reg_alpha=2.0,
                    reg_lambda=3.0,
                    min_child_weight=10,
                    random_state=42,
                    n_jobs=-1
                )),
                ('lgbm', LGBMClassifier(
                    n_estimators=lgbm_n_est,
                    max_depth=4,
                    learning_rate=0.03,
                    subsample=0.6,
                    reg_alpha=2.0,
                    reg_lambda=2.0,
                    min_child_samples=60,
                    random_state=42,
                    verbose=-1
                ))
            ]
            
            model = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_params = study.best_params
        estimators = [
            ('xgb', XGBClassifier(
                n_estimators=best_params['xgb_n_estimators'],
                max_depth=best_params['xgb_max_depth'],
                learning_rate=0.03,
                subsample=0.6,
                colsample_bytree=0.6,
                reg_alpha=2.0,
                reg_lambda=3.0,
                min_child_weight=10,
                random_state=42,
                n_jobs=-1
            )),
            ('lgbm', LGBMClassifier(
                n_estimators=best_params['lgbm_n_estimators'],
                max_depth=4,
                learning_rate=0.03,
                subsample=0.6,
                reg_alpha=2.0,
                reg_lambda=2.0,
                min_child_samples=60,
                random_state=42,
                verbose=-1
            ))
        ]
        
        best_model = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Voting Hard] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model
    
    @staticmethod
    def voting_soft(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            xgb_n_est = trial.suggest_int('xgb_n_estimators', 100, 200)
            xgb_depth = trial.suggest_int('xgb_max_depth', 3, 5)
            
            estimators = [
                ('xgb', XGBClassifier(
                    n_estimators=xgb_n_est,
                    max_depth=xgb_depth,
                    learning_rate=0.03,
                    subsample=0.6,
                    colsample_bytree=0.6,
                    reg_alpha=2.0,
                    reg_lambda=3.0,
                    min_child_weight=10,
                    random_state=42,
                    n_jobs=-1
                )),
                ('lgbm', LGBMClassifier(
                    n_estimators=120,
                    max_depth=4,
                    learning_rate=0.03,
                    subsample=0.6,
                    reg_alpha=2.0,
                    reg_lambda=2.0,
                    min_child_samples=60,
                    random_state=42,
                    verbose=-1
                ))
            ]
            
            model = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
            model.fit(X_train, y_train)
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=6)
        )
        
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_params = study.best_params
        estimators = [
            ('xgb', XGBClassifier(
                n_estimators=best_params['xgb_n_estimators'],
                max_depth=best_params['xgb_max_depth'],
                learning_rate=0.03,
                subsample=0.6,
                colsample_bytree=0.6,
                reg_alpha=2.0,
                reg_lambda=3.0,
                min_child_weight=10,
                random_state=42,
                n_jobs=-1
            )),
            ('lgbm', LGBMClassifier(
                n_estimators=120,
                max_depth=4,
                learning_rate=0.03,
                subsample=0.6,
                reg_alpha=2.0,
                reg_lambda=2.0,
                min_child_samples=60,
                random_state=42,
                verbose=-1
            ))
        ]
        
        best_model = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        train_acc = best_model.score(X_train, y_train)
        val_acc = best_model.score(X_val, y_val)
        print(f"[Voting Soft] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return best_model

    
    
    
    
    @staticmethod
    def lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 32, 80, step=16)
            units2 = trial.suggest_int('units2', 16, 48, step=16)
            dropout = trial.suggest_float('dropout', 0.35, 0.55)
            l2_reg = trial.suggest_float('l2_reg', 0.01, 0.15, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.002, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                LSTM(units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=8, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            Dense(16, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def bilstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 24, 64, step=16)
            units2 = trial.suggest_int('units2', 12, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.002, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                Bidirectional(LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0), input_shape=input_shape),
                Dropout(dropout),
                BatchNormalization(),
                Bidirectional(LSTM(units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0)),
                Dropout(dropout),
                BatchNormalization(),
                Dense(12, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            Bidirectional(LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0), input_shape=input_shape),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            Bidirectional(LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0)),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[BiLSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def gru(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 32, 96, step=16)
            units2 = trial.suggest_int('units2', 16, 56, step=16)
            dropout = trial.suggest_float('dropout', 0.35, 0.55)
            l2_reg = trial.suggest_float('l2_reg', 0.01, 0.15, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.002, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                GRU(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                GRU(units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=8, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            GRU(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            GRU(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            Dense(16, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[GRU] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def stacked_lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 32, 72, step=16)
            units2 = trial.suggest_int('units2', 24, 48, step=12)
            units3 = trial.suggest_int('units3', 12, 32, step=8)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.00005, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                LSTM(units2, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                LSTM(units3, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                Dense(12, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=0.5), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            LSTM(best_params['units3'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=0.5), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-8, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[Stacked LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def vmd_hybrid(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            n_filters = trial.suggest_int('n_filters', 16, 32, step=8)
            num_heads = trial.suggest_int('num_heads', 2, 4)
            key_dim = trial.suggest_int('key_dim', 16, 32, step=8)
            ff_dim = trial.suggest_int('ff_dim', 48, 96, step=24)
            dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.5)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.1, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            inputs = Input(shape=input_shape)
            x = Conv1D(n_filters, 1, padding='same', kernel_regularizer=l2(l2_reg))(inputs)
            x = BatchNormalization()(x)

            low_freq = AveragePooling1D(pool_size=5, strides=1, padding='same')(x)
            low_freq = Conv1D(n_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(low_freq)

            mid_freq = x - low_freq
            mid_freq = Conv1D(n_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(mid_freq)

            high_freq = x - low_freq - mid_freq
            high_freq = Conv1D(n_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(high_freq)

            x = Concatenate()([low_freq, mid_freq, high_freq])
            x = BatchNormalization()(x)
            x = Dropout(dropout_rate)(x)

            attn = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout_rate, kernel_regularizer=l2(l2_reg))(x, x)
            attn = Dropout(dropout_rate)(attn)
            x = LayerNormalization(epsilon=1e-6)(x + attn)

            ff = Dense(ff_dim, activation='gelu', kernel_regularizer=l2(l2_reg))(x)
            ff = Dropout(dropout_rate)(ff)
            ff = Dense(n_filters * 3, kernel_regularizer=l2(l2_reg))(ff)
            ff = Dropout(dropout_rate)(ff)
            x = LayerNormalization(epsilon=1e-6)(x + ff)

            x = GlobalAveragePooling1D()(x)
            x = Dense(24, activation='relu', kernel_regularizer=l2(l2_reg))(x)
            x = BatchNormalization()(x)
            x = Dropout(dropout_rate)(x)
            outputs = Dense(1, activation='sigmoid')(x)

            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        inputs = Input(shape=input_shape)
        x = Conv1D(best_params['n_filters'], 1, padding='same', kernel_regularizer=l2(best_params['l2_reg']))(inputs)
        x = BatchNormalization()(x)

        low_freq = AveragePooling1D(pool_size=5, strides=1, padding='same')(x)
        low_freq = Conv1D(best_params['n_filters'], 3, activation='relu', padding='same', kernel_regularizer=l2(best_params['l2_reg']))(low_freq)

        mid_freq = x - low_freq
        mid_freq = Conv1D(best_params['n_filters'], 3, activation='relu', padding='same', kernel_regularizer=l2(best_params['l2_reg']))(mid_freq)

        high_freq = x - low_freq - mid_freq
        high_freq = Conv1D(best_params['n_filters'], 3, activation='relu', padding='same', kernel_regularizer=l2(best_params['l2_reg']))(high_freq)

        x = Concatenate()([low_freq, mid_freq, high_freq])
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout_rate'])(x)

        attn = MultiHeadAttention(num_heads=best_params['num_heads'], key_dim=best_params['key_dim'], dropout=best_params['dropout_rate'], kernel_regularizer=l2(best_params['l2_reg']))(x, x)
        attn = Dropout(best_params['dropout_rate'])(attn)
        x = LayerNormalization(epsilon=1e-6)(x + attn)

        ff = Dense(best_params['ff_dim'], activation='gelu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        ff = Dropout(best_params['dropout_rate'])(ff)
        ff = Dense(best_params['n_filters'] * 3, kernel_regularizer=l2(best_params['l2_reg']))(ff)
        ff = Dropout(best_params['dropout_rate'])(ff)
        x = LayerNormalization(epsilon=1e-6)(x + ff)

        x = GlobalAveragePooling1D()(x)
        x = Dense(24, activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout_rate'])(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[VMD-Hybrid] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def dtw_lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 48, 80, step=16)
            units2 = trial.suggest_int('units2', 32, 56, step=12)
            units3 = trial.suggest_int('units3', 16, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                LSTM(units2, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                LSTM(units3, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                Dropout(dropout),
                Dense(12, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            LSTM(best_params['units3'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            Dropout(best_params['dropout']),
            Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[DTW-LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def emd_lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            freq_units = trial.suggest_int('freq_units', 24, 48, step=12)
            units1 = trial.suggest_int('units1', 32, 64, step=16)
            units2 = trial.suggest_int('units2', 16, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            inputs = Input(shape=input_shape)

            low_freq = AveragePooling1D(pool_size=5, strides=1, padding='same')(inputs)
            low_freq = LSTM(freq_units, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(low_freq)

            high_freq = inputs - AveragePooling1D(pool_size=5, strides=1, padding='same')(inputs)
            high_freq = LSTM(freq_units, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(high_freq)

            x = Concatenate()([low_freq, high_freq])
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)

            x = LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(x)
            x = BatchNormalization()(x)

            x = LSTM(units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(x)
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)

            x = Dense(12, activation='relu', kernel_regularizer=l2(l2_reg))(x)
            x = Dropout(dropout)(x)
            outputs = Dense(1, activation='sigmoid')(x)

            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        inputs = Input(shape=input_shape)

        low_freq = AveragePooling1D(pool_size=5, strides=1, padding='same')(inputs)
        low_freq = LSTM(best_params['freq_units'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(low_freq)

        high_freq = inputs - AveragePooling1D(pool_size=5, strides=1, padding='same')(inputs)
        high_freq = LSTM(best_params['freq_units'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(high_freq)

        x = Concatenate()([low_freq, high_freq])
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout'])(x)

        x = LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(x)
        x = BatchNormalization()(x)

        x = LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(x)
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout'])(x)

        x = Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        x = Dropout(best_params['dropout'])(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[EMD-LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def hybrid_lstm_gru(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 48, 80, step=16)
            units2 = trial.suggest_int('units2', 32, 56, step=12)
            units3 = trial.suggest_int('units3', 16, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), recurrent_dropout=0.0),
                Dropout(dropout),
                BatchNormalization(),
                GRU(units2, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                LSTM(units3, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                Dropout(dropout),
                Dense(12, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), recurrent_dropout=0.0),
            Dropout(best_params['dropout']),
            BatchNormalization(),
            GRU(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            LSTM(best_params['units3'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            Dropout(best_params['dropout']),
            Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[Hybrid LSTM-GRU] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def residual_lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            units1 = trial.suggest_int('units1', 48, 80, step=16)
            units2 = trial.suggest_int('units2', 16, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.2, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            inputs = Input(shape=input_shape)

            x = LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(inputs)
            x = BatchNormalization()(x)

            lstm_out = LSTM(units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(x)
            lstm_out = BatchNormalization()(lstm_out)
            x = Add()([x, lstm_out])
            x = Dropout(dropout)(x)

            x = LSTM(units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(x)
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)

            x = Dense(12, activation='relu', kernel_regularizer=l2(l2_reg))(x)
            x = Dropout(dropout)(x)
            outputs = Dense(1, activation='sigmoid')(x)

            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        inputs = Input(shape=input_shape)

        x = LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(inputs)
        x = BatchNormalization()(x)

        lstm_out = LSTM(best_params['units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(x)
        lstm_out = BatchNormalization()(lstm_out)
        x = Add()([x, lstm_out])
        x = Dropout(best_params['dropout'])(x)

        x = LSTM(best_params['units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(x)
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout'])(x)

        x = Dense(12, activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        x = Dropout(best_params['dropout'])(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[Residual LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def tcn(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            n_filters = trial.suggest_int('n_filters', 16, 48, step=16)
            kernel_size = trial.suggest_int('kernel_size', 3, 5)
            n_stacks = trial.suggest_int('n_stacks', 1, 2)
            dropout = trial.suggest_float('dropout', 0.3, 0.5)
            l2_reg = trial.suggest_float('l2_reg', 0.01, 0.1, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.002, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            inputs = Input(shape=input_shape)
            x = inputs

            for stack in range(n_stacks):
                dilation_rates = [2**i for i in range(4)]

                for dilation_rate in dilation_rates:
                    residual = x

                    conv1 = Conv1D(filters=n_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal', activation='relu', kernel_regularizer=l2(l2_reg))(x)
                    conv1 = BatchNormalization()(conv1)
                    conv1 = Dropout(dropout)(conv1)

                    conv2 = Conv1D(filters=n_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal', activation='relu', kernel_regularizer=l2(l2_reg))(conv1)
                    conv2 = BatchNormalization()(conv2)
                    conv2 = Dropout(dropout)(conv2)

                    if residual.shape[-1] != n_filters:
                        residual = Conv1D(n_filters, 1, padding='same')(residual)

                    x = Add()([residual, conv2])

            x = GlobalAveragePooling1D()(x)
            x = Dense(24, activation='relu', kernel_regularizer=l2(l2_reg))(x)
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)
            outputs = Dense(1, activation='sigmoid')(x)

            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        inputs = Input(shape=input_shape)
        x = inputs

        for stack in range(best_params['n_stacks']):
            dilation_rates = [2**i for i in range(4)]

            for dilation_rate in dilation_rates:
                residual = x

                conv1 = Conv1D(filters=best_params['n_filters'], kernel_size=best_params['kernel_size'], dilation_rate=dilation_rate, padding='causal', activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
                conv1 = BatchNormalization()(conv1)
                conv1 = Dropout(best_params['dropout'])(conv1)

                conv2 = Conv1D(filters=best_params['n_filters'], kernel_size=best_params['kernel_size'], dilation_rate=dilation_rate, padding='causal', activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(conv1)
                conv2 = BatchNormalization()(conv2)
                conv2 = Dropout(best_params['dropout'])(conv2)

                if residual.shape[-1] != best_params['n_filters']:
                    residual = Conv1D(best_params['n_filters'], 1, padding='same')(residual)

                x = Add()([residual, conv2])

        x = GlobalAveragePooling1D()(x)
        x = Dense(24, activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout'])(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[TCN] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def cnn_lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            cnn_filters = trial.suggest_int('cnn_filters', 16, 48, step=16)
            cnn_kernel = trial.suggest_int('cnn_kernel', 3, 5)
            lstm_units1 = trial.suggest_int('lstm_units1', 32, 64, step=16)
            lstm_units2 = trial.suggest_int('lstm_units2', 16, 40, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.15, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            model = Sequential([
                Conv1D(filters=cnn_filters, kernel_size=cnn_kernel, activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
                BatchNormalization(),
                Dropout(dropout),
                Conv1D(filters=cnn_filters, kernel_size=cnn_kernel, activation='relu', padding='same', kernel_regularizer=l2(l2_reg)),
                BatchNormalization(),
                MaxPooling1D(pool_size=2),
                Dropout(dropout),
                LSTM(lstm_units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                LSTM(lstm_units2, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0),
                BatchNormalization(),
                Dropout(dropout),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        model = Sequential([
            Conv1D(filters=best_params['cnn_filters'], kernel_size=best_params['cnn_kernel'], activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(best_params['l2_reg'])),
            BatchNormalization(),
            Dropout(best_params['dropout']),
            Conv1D(filters=best_params['cnn_filters'], kernel_size=best_params['cnn_kernel'], activation='relu', padding='same', kernel_regularizer=l2(best_params['l2_reg'])),
            BatchNormalization(),
            MaxPooling1D(pool_size=2),
            Dropout(best_params['dropout']),
            LSTM(best_params['lstm_units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            LSTM(best_params['lstm_units2'], activation='tanh', recurrent_activation='sigmoid', kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0),
            BatchNormalization(),
            Dropout(best_params['dropout']),
            Dense(16, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])),
            Dropout(best_params['dropout']),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[CNN-LSTM] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


    @staticmethod
    def lstm_attention(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            lstm_units1 = trial.suggest_int('lstm_units1', 48, 80, step=16)
            lstm_units2 = trial.suggest_int('lstm_units2', 32, 56, step=12)
            dropout = trial.suggest_float('dropout', 0.4, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 0.02, 0.15, log=True)
            learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001, log=True)

            X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

            inputs = Input(shape=input_shape)

            lstm_out = LSTM(lstm_units1, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg * 0.5), dropout=dropout, recurrent_dropout=0.0)(inputs)
            lstm_out = BatchNormalization()(lstm_out)

            attention = Dense(1, activation='tanh', kernel_regularizer=l2(l2_reg))(lstm_out)
            attention = Flatten()(attention)
            attention = Activation('softmax')(attention)
            attention = RepeatVector(lstm_units1)(attention)
            attention = Permute([2, 1])(attention)

            attended = Multiply()([lstm_out, attention])
            attended = Lambda(lambda x: tf.reduce_sum(x, axis=1))(attended)
            attended = BatchNormalization()(attended)
            attended = Dropout(dropout)(attended)

            x = Dense(lstm_units2, activation='relu', kernel_regularizer=l2(l2_reg))(attended)
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)

            x = Dense(16, activation='relu', kernel_regularizer=l2(l2_reg))(x)
            x = Dropout(dropout)(x)
            outputs = Dense(1, activation='sigmoid')(x)

            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
            early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, mode='min')
            history = model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32, callbacks=[early_stop], verbose=0)

            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            return val_accuracy

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=3), pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=3))
        study.optimize(objective, n_trials=6, show_progress_bar=False)

        best_params = study.best_params
        X_aug = TimeSeriesAugmentation.jittering(X_train, sigma=0.015)

        inputs = Input(shape=input_shape)

        lstm_out = LSTM(best_params['lstm_units1'], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(best_params['l2_reg']), recurrent_regularizer=l2(best_params['l2_reg'] * 0.5), dropout=best_params['dropout'], recurrent_dropout=0.0)(inputs)
        lstm_out = BatchNormalization()(lstm_out)

        attention = Dense(1, activation='tanh', kernel_regularizer=l2(best_params['l2_reg']))(lstm_out)
        attention = Flatten()(attention)
        attention = Activation('softmax')(attention)
        attention = RepeatVector(best_params['lstm_units1'])(attention)
        attention = Permute([2, 1])(attention)

        attended = Multiply()([lstm_out, attention])
        attended = Lambda(lambda x: tf.reduce_sum(x, axis=1))(attended)
        attended = BatchNormalization()(attended)
        attended = Dropout(best_params['dropout'])(attended)

        x = Dense(best_params['lstm_units2'], activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(attended)
        x = BatchNormalization()(x)
        x = Dropout(best_params['dropout'])(x)

        x = Dense(16, activation='relu', kernel_regularizer=l2(best_params['l2_reg']))(x)
        x = Dropout(best_params['dropout'])(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'], clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=1e-4, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-7, mode='min', verbose=0)

        model.fit(X_aug, y_train, validation_data=(X_val, y_val), epochs=60, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        print(f"[LSTM-Attention] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        return model


## dropout 설정때매 오류난다 캐서 그거 변경 하기 전 버전

In [7]:
# ML Models (19개)
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 2, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 3, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True},
    {'index': 4, 'name': 'SVM', 'func': DirectionModels.svm, 'needs_val': True},
    {'index': 5, 'name': 'LogisticRegression', 'func': DirectionModels.logistic_regression, 'needs_val': True},
    {'index': 6, 'name': 'NaiveBayes', 'func': DirectionModels.naive_bayes, 'needs_val': True},
    {'index': 7, 'name': 'KNN', 'func': DirectionModels.knn, 'needs_val': True},
    {'index': 8, 'name': 'AdaBoost', 'func': DirectionModels.adaboost, 'needs_val': True},
    {'index': 9, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 10, 'name': 'DecisionTree', 'func': DirectionModels.decision_tree, 'needs_val': True},
    {'index': 11, 'name': 'ExtraTrees', 'func': DirectionModels.extra_trees, 'needs_val': True},
    {'index': 12, 'name': 'Bagging', 'func': DirectionModels.bagging, 'needs_val': True},
    {'index': 13, 'name': 'GradientBoosting', 'func': DirectionModels.gradient_boosting, 'needs_val': True},
    {'index': 14, 'name': 'HistGradientBoosting', 'func': DirectionModels.histgradient_boosting, 'needs_val': True},
    {'index': 15, 'name': 'StackingEnsemble', 'func': DirectionModels.stacking_ensemble, 'needs_val': True},
    {'index': 16, 'name': 'VotingHard', 'func': DirectionModels.voting_hard, 'needs_val': True},
    {'index': 17, 'name': 'VotingSoft', 'func': DirectionModels.voting_soft, 'needs_val': True},
    {'index': 18, 'name': 'MLP', 'func': DirectionModels.mlp, 'needs_val': True},
    #{'index': 30, 'name': 'TabNet', 'func': DirectionModels.tabnet, 'needs_val': True},
]

# DL Models (11개)
DL_MODELS_CLASSIFICATION = [
    {'index': 19, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
    {'index': 20, 'name': 'BiLSTM', 'func': DirectionModels.bilstm, 'needs_val': True},
    {'index': 21, 'name': 'GRU', 'func': DirectionModels.gru, 'needs_val': True},
    {'index': 22, 'name': 'TCN', 'func': DirectionModels.tcn, 'needs_val': True},
    {'index': 23, 'name': 'CNN_LSTM', 'func': DirectionModels.cnn_lstm, 'needs_val': True},
    {'index': 24, 'name': 'LSTM_Attention', 'func': DirectionModels.lstm_attention, 'needs_val': True},
    {'index': 25, 'name': 'DTW_LSTM', 'func': DirectionModels.dtw_lstm, 'needs_val': True},
    {'index': 26, 'name': 'VMD_Hybrid', 'func': DirectionModels.vmd_hybrid, 'needs_val': True},
    {'index': 27, 'name': 'EMD_LSTM', 'func': DirectionModels.emd_lstm, 'needs_val': True},
    {'index': 28, 'name': 'Hybrid_LSTM_GRU', 'func': DirectionModels.hybrid_lstm_gru, 'needs_val': True},
    {'index': 29, 'name': 'Residual_LSTM', 'func': DirectionModels.residual_lstm, 'needs_val': True},
]


In [10]:
import time
from datetime import datetime

# timestamp = datetime.now().strftime("%Y-%m-%d")
timestamp='2025-10-26'
RESULT_DIR = os.path.join("model_results", timestamp)
os.makedirs(RESULT_DIR, exist_ok=True)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)


class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.predictions = {}
        self.models = {} if save_models else None
        self.save_models = save_models
    
    def _predict_model(self, model, X):
        pred = model.predict(X)
        if isinstance(pred, list):
            cleaned = []
            for p in pred:
                if isinstance(p, np.ndarray):
                    cleaned.append(p.squeeze() if p.shape[-1] == 1 else p)
                else:
                    cleaned.append(p)
            return cleaned
        else:
            return pred.squeeze() if pred.shape[-1] == 1 else pred
    
    def evaluate_classification_model(self, model, X_train, y_train, X_val, y_val, 
                                     X_test, y_test, test_returns, test_dates, model_name,
                                     is_deep_learning=False):
        
        train_pred = self._predict_model(model, X_train)
        val_pred = self._predict_model(model, X_val)
        test_pred = self._predict_model(model, X_test)
        
        test_pred_proba = None
        if is_deep_learning:
            test_pred_proba = test_pred.copy()
            if isinstance(train_pred, list):
                train_pred = train_pred[0]
                val_pred = val_pred[0]
                test_pred = test_pred[0]
                test_pred_proba = test_pred_proba[0] if isinstance(test_pred_proba, list) else test_pred_proba
            train_pred = (train_pred > 0.5).astype(int).ravel()
            val_pred = (val_pred > 0.5).astype(int).ravel()
            test_pred = (test_pred > 0.5).astype(int).ravel()
        else:
            if hasattr(model, 'predict_proba'):
                test_pred_proba = model.predict_proba(X_test)
        
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        test_acc = accuracy_score(y_test, test_pred)
        test_prec = precision_score(y_test, test_pred, zero_division=0)
        test_rec = recall_score(y_test, test_pred, zero_division=0)
        test_f1 = f1_score(y_test, test_pred, zero_division=0)
        test_roc_auc = roc_auc_score(y_test, test_pred)
        
        self._save_predictions(model_name, test_pred, test_pred_proba, y_test, test_returns, test_dates)
        
        if self.save_models and self.models is not None:
            self.models[model_name] = model
        
        self.results.append({
            'Model': model_name,
            'Train_Accuracy': train_acc,
            'Val_Accuracy': val_acc,
            'Test_Accuracy': test_acc,
            'Test_Precision': test_prec,
            'Test_Recall': test_rec,
            'Test_F1': test_f1,
            'Test_AUC_ROC': test_roc_auc
        })
        
        del train_pred, val_pred, test_pred, test_pred_proba
        gc.collect()
        
        return self.results[-1]
    
    def _save_predictions(self, model_name, pred_direction, pred_proba, actual_direction, actual_returns, dates):
        if pred_proba is not None:
            if pred_proba.ndim == 2 and pred_proba.shape[1] == 2:
                pred_proba_up = pred_proba[:, 1]
                pred_proba_down = pred_proba[:, 0]
            else:
                pred_proba_up = pred_proba.ravel()
                pred_proba_down = 1 - pred_proba_up
        else:
            pred_proba_up = np.where(pred_direction == 1, 0.9, 0.1)
            pred_proba_down = 1 - pred_proba_up
        
        max_proba = np.maximum(pred_proba_up, pred_proba_down)
        confidence = np.abs(pred_proba_up - 0.5) * 2
        
        predictions_df = pd.DataFrame({
            'date': dates,
            'actual_direction': actual_direction,
            'actual_return': actual_returns,
            'pred_direction': pred_direction,
            'pred_proba_up': pred_proba_up,
            'pred_proba_down': pred_proba_down,
            'max_proba': max_proba,
            'confidence': confidence,
            'correct': (pred_direction == actual_direction).astype(int)
        })
        
        self.predictions[model_name] = predictions_df
    
    def get_summary_dataframe(self):
        return pd.DataFrame(self.results)
    
    def get_predictions_dict(self):
        return self.predictions
    
    def get_models_dict(self):
        return self.models if self.models is not None else {}


class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    @staticmethod
    def create_sequences(X, y, lookback):
        Xs, ys = [], []
        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y.iloc[i] if hasattr(y, 'iloc') else y[i])
        X_arr = np.array(Xs)
        y_arr = np.array(ys)
        del Xs, ys
        gc.collect()
        return X_arr, y_arr
    
    @staticmethod
    def clear_memory():
        keras.backend.clear_session()
        try:
            tf.compat.v1.reset_default_graph()
        except:
            pass
        gc.collect()
        time.sleep(0.5)
    
    def train_ml_model(self, model_config, X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates, task='classification'):
        model = None
        try:
            if model_config.get('needs_val', False):
                model = model_config['func'](X_train, y_train, X_val, y_val)
            else:
                model = model_config['func'](X_train, y_train)
            
            is_mlp = (model_config['name'] == 'MLP')
            
            self.evaluator.evaluate_classification_model(
                model, X_train, y_train, X_val, y_val, X_test, y_test,
                test_returns, test_dates, model_config['name'], is_deep_learning=is_mlp
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                if is_mlp:
                    self.clear_memory()
                else:
                    gc.collect()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            if model_config.get('name') == 'MLP':
                self.clear_memory()
            else:
                gc.collect()
    
    def train_dl_model(self, model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, test_returns_seq, test_dates_seq, input_shape, task='classification'):
        model = None
        try:
            self.clear_memory()
            
            model = model_config['func'](X_train_seq, y_train_seq, X_val_seq, y_val_seq, input_shape)
            
            self.evaluator.evaluate_classification_model(
                model, X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                X_test_seq, y_test_seq, test_returns_seq, test_dates_seq,
                model_config['name'], is_deep_learning=True
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                self.clear_memory()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            self.clear_memory()


def train_all_models(X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates, evaluator, lookback=30, ml_models=None, dl_models=None):
    trainer = ModelTrainer(evaluator, lookback)
    
    ml_success = 0
    for model_config in ml_models:
        if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates):
            ml_success += 1
        gc.collect()
    
    trainer.clear_memory()
    
    X_train_seq, y_train_seq = trainer.create_sequences(X_train, y_train, lookback)
    X_val_seq, y_val_seq = trainer.create_sequences(X_val, y_val, lookback)
    X_test_seq, y_test_seq = trainer.create_sequences(X_test, y_test, lookback)
    test_returns_seq = test_returns[lookback:]
    test_dates_seq = test_dates[lookback:]
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    
    dl_success = 0
    for model_config in dl_models:
        trainer.clear_memory()
        
        if model_config['name'] in ['TabNet', 'StackingEnsemble', 'VotingHard', 'VotingSoft']:
            if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates):
                dl_success += 1
        else:
            if trainer.train_dl_model(model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, test_returns_seq, test_dates_seq, input_shape):
                dl_success += 1
        
        gc.collect()
    
    del X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, test_returns_seq, test_dates_seq
    trainer.clear_memory()
    
    return ml_success + dl_success


GPU memory growth enabled for 1 GPU(s)


In [ ]:


import os
import pandas as pd
import numpy as np
import joblib
import gc
import time

def save_raw_data_once(result, target_name, split_method):
    raw_dir = os.path.join(RESULT_DIR, "raw_data", target_name, split_method)
    os.makedirs(raw_dir, exist_ok=True)
    
    if split_method == 'tvt':
        for split in ['train', 'val', 'test']:
            df = pd.DataFrame(result[split]['X_raw'], columns=result['stats']['selected_features'])
            df['date'] = result[split]['dates']
            for col in result[split]['y'].columns:
                df[col] = result[split]['y'][col].values
            df.to_csv(os.path.join(raw_dir, f"{split}_raw.csv"), index=False, encoding='utf-8-sig')
        
        del result['train']['X_raw'], result['val']['X_raw'], result['test']['X_raw']
    else:
        for fold_idx, fold in enumerate(result, start=1):
            fold_type = fold['stats']['fold_type']
            fold_dir = os.path.join(raw_dir, f"fold_{fold_idx}_{fold_type}")
            os.makedirs(fold_dir, exist_ok=True)
            
            for split in ['train', 'val', 'test']:
                df = pd.DataFrame(fold[split]['X_raw'], columns=fold['stats']['selected_features'])
                df['date'] = fold[split]['dates']
                for col in fold[split]['y'].columns:
                    df[col] = fold[split]['y'][col].values
                df.to_csv(os.path.join(fold_dir, f"{split}_raw.csv"), index=False, encoding='utf-8-sig')
            
            del fold['train']['X_raw'], fold['val']['X_raw'], fold['test']['X_raw']
    
    print(f"Saved raw data CSVs to {raw_dir}")
    gc.collect()

def check_fold_completed(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    summary_path = os.path.join(fold_dir, "fold_summary.csv")
    return os.path.exists(summary_path)

def save_fold_results(fold_idx, fold_type, evaluator, target_name):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    os.makedirs(fold_dir, exist_ok=True)
    
    models = evaluator.get_models_dict()
    for model_name, model_obj in models.items():
        is_dl = model_name in ['LSTM', 'BiLSTM', 'GRU', 'TCN', 'CNN_LSTM', 
                               'LSTM_Attention', 'DTW_LSTM', 'VMD_Hybrid', 
                               'EMD_LSTM', 'Hybrid_LSTM_GRU', 'Residual_LSTM', 
                               'MLP', 'Stacked_LSTM']
        
        if is_dl:
            model_path = os.path.join(fold_dir, f"{model_name}.h5")
            model_obj.save(model_path)
        else:
            model_path = os.path.join(fold_dir, f"{model_name}.pkl")
            joblib.dump(model_obj, model_path, compress=3)
    
    fold_predictions = evaluator.get_predictions_dict()
    for model_name, pred_df in fold_predictions.items():
        pred_path = os.path.join(fold_dir, f"{model_name}_predictions.csv")
        pred_df.to_csv(pred_path, index=False, encoding='utf-8-sig')
    
    fold_summary = evaluator.get_summary_dataframe()
    summary_path = os.path.join(fold_dir, "fold_summary.csv")
    fold_summary.to_csv(summary_path, index=False, encoding='utf-8-sig')
    
    print(f"Saved fold {fold_idx} ({fold_type}) results to {fold_dir}")
    
    return fold_summary, fold_predictions

def load_fold_results(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    summary_path = os.path.join(fold_dir, "fold_summary.csv")
    
    if not os.path.exists(summary_path):
        return None, None
    
    fold_summary = pd.read_csv(summary_path)
    
    fold_predictions = {}
    for file in os.listdir(fold_dir):
        if file.endswith('_predictions.csv'):
            model_name = file.replace('_predictions.csv', '')
            fold_predictions[model_name] = pd.read_csv(os.path.join(fold_dir, file))
    
    return fold_summary, fold_predictions

def save_walk_forward_summary(all_fold_results, target_name):
    detailed_results = []
    for fold_idx, (fold_df, fold_type) in enumerate(all_fold_results, start=1):
        fold_df_copy = fold_df.copy()
        fold_df_copy.insert(0, 'Fold', fold_idx)
        fold_df_copy.insert(1, 'fold_type', fold_type)
        detailed_results.append(fold_df_copy)
    
    detailed_df = pd.concat(detailed_results, ignore_index=True)
    detailed_df.to_csv(os.path.join(RESULT_DIR, f"{target_name}_all_folds_detailed.csv"), index=False, encoding='utf-8-sig')
    
    wf_data = detailed_df[detailed_df['fold_type'] == 'walk_forward'].copy()
    
    numeric_cols = [col for col in wf_data.select_dtypes(include=[np.number]).columns if col != 'Fold']
    
    avg_results = []
    for model in wf_data['Model'].unique():
        avg_row = {'Model': model}
        model_wf = wf_data[wf_data['Model'] == model]
        for col in numeric_cols:
            if col in model_wf.columns:
                avg_row[f'{col}_Mean'] = model_wf[col].mean()
                avg_row[f'{col}_Std'] = model_wf[col].std()
        avg_results.append(avg_row)
    
    avg_df = pd.DataFrame(avg_results)
    if 'Test_Accuracy_Mean' in avg_df.columns:
        avg_df = avg_df.sort_values(by='Test_Accuracy_Mean', ascending=False).reset_index(drop=True)
    
    avg_df.to_csv(os.path.join(RESULT_DIR, f"{target_name}_walk_forward_average.csv"), index=False, encoding='utf-8-sig')
    
    print(f"Saved walk-forward summary to {RESULT_DIR}")

def save_tvt_results(summary_df, predictions_dict, target_name):
    summary_df.to_csv(os.path.join(RESULT_DIR, f"{target_name}_tvt_summary.csv"), index=False, encoding='utf-8-sig')
    
    if predictions_dict:
        pred_dir = os.path.join(RESULT_DIR, "predictions", f"{target_name}_tvt")
        os.makedirs(pred_dir, exist_ok=True)
        
        for model_name, pred_df in predictions_dict.items():
            pred_df.to_csv(os.path.join(pred_dir, f"{model_name}.csv"), index=False, encoding='utf-8-sig')

target_cases = [{'name': 'direction', 'target_type': 'direction', 'outputs': ['next_direction']}]
split_methods = [{'name': 'walk_forward', 'method': 'walk_forward'}]

all_results = {}

for target_case in target_cases:
    for split_method in split_methods:
        
        result = build_complete_pipeline_corrected(
            df_merged, TRAIN_START_DATE,
            method=split_method['method'],
            target_type=target_case['target_type'],
            test_start_date='2025-01-01'
        )
        
        save_raw_data_once(result, target_case['name'], split_method['method'])
        
        if split_method['method'] == 'tvt':
            X_train = result['train']['X_robust']
            X_val = result['val']['X_robust']
            X_test = result['test']['X_robust']
            test_returns = result['test']['y']['next_log_return'].values
            test_dates = result['test']['dates'].values
            y_train = result['train']['y'][target_case['outputs'][0]].values
            y_val = result['val']['y'][target_case['outputs'][0]].values
            y_test = result['test']['y'][target_case['outputs'][0]].values
            
            evaluator = ModelEvaluator(save_models=False)
            train_all_models(X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates, evaluator, ml_models=ML_MODELS_CLASSIFICATION, dl_models=DL_MODELS_CLASSIFICATION)
            
            summary_df = evaluator.get_summary_dataframe()
            predictions_dict = evaluator.get_predictions_dict()
            
            save_tvt_results(summary_df, predictions_dict, target_case['name'])
            all_results[f"{target_case['name']}_tvt"] = summary_df
            
            del evaluator
            keras.backend.clear_session()
            gc.collect()
        
        else:
            fold_results = []
            
            for fold_idx, fold in enumerate(result, start=1):
                fold_type = fold['stats']['fold_type']
                print(f"\n=== Fold {fold_idx} ({fold_type}) ===")
                
                if check_fold_completed(target_case['name'], fold_idx, fold_type):
                    print(f"Fold {fold_idx} already completed. Loading results...")
                    fold_summary, fold_pred_dict = load_fold_results(target_case['name'], fold_idx, fold_type)
                    if fold_summary is not None:
                        fold_results.append((fold_summary, fold_type))
                        print(f"Loaded results for Fold {fold_idx}")
                        continue
                
                X_train = fold['train']['X_robust']
                X_val = fold['val']['X_robust']
                X_test = fold['test']['X_robust']
                test_returns = fold['test']['y']['next_log_return'].values
                test_dates = fold['test']['dates'].values
                y_train = fold['train']['y'][target_case['outputs'][0]].values
                y_val = fold['val']['y'][target_case['outputs'][0]].values
                y_test = fold['test']['y'][target_case['outputs'][0]].values
                
                evaluator = ModelEvaluator(save_models=True)
                
                try:
                    train_all_models(X_train, y_train, X_val, y_val, X_test, y_test, test_returns, test_dates, evaluator, ml_models=ML_MODELS_CLASSIFICATION, dl_models=DL_MODELS_CLASSIFICATION)
                    
                    fold_summary, fold_pred_dict = save_fold_results(fold_idx, fold_type, evaluator, target_case['name'])
                    fold_results.append((fold_summary, fold_type))
                    
                    del evaluator
                except Exception as e:
                    print(f"Fold {fold_idx} failed: {e}")
                    if 'evaluator' in locals():
                        del evaluator
                finally:
                    keras.backend.clear_session()
                    try:
                        tf.compat.v1.reset_default_graph()
                    except:
                        pass
                    gc.collect()
                    time.sleep(1)
            
            save_walk_forward_summary(fold_results, target_case['name'])
            
            keras.backend.clear_session()
            gc.collect()

print(f"\nComplete! Results saved to: {RESULT_DIR}")



Reverse Rolling Walk-Forward Configuration 
Total: 2121 days
Rolling train size: 800 days (FIXED)
Val: 150 days | Test: 150 days
Gap: 7 days | Step: 150 days (BACKWARD)
Target: 7 walk-forward + 1 final holdout

Fold 1 (walk_forward_rolling)
  Train:  800d  2020-04-17 ~ 2022-06-25
  Val:    150d  2022-07-03 ~ 2022-11-29
  Test:   150d  2022-12-07 ~ 2023-05-05

Fold 2 (walk_forward_rolling)
  Train:  800d  2020-09-14 ~ 2022-11-22
  Val:    150d  2022-11-30 ~ 2023-04-28
  Test:   150d  2023-05-06 ~ 2023-10-02

Fold 3 (walk_forward_rolling)
  Train:  800d  2021-02-11 ~ 2023-04-21
  Val:    150d  2023-04-29 ~ 2023-09-25
  Test:   150d  2023-10-03 ~ 2024-02-29

Fold 4 (walk_forward_rolling)
  Train:  800d  2021-07-11 ~ 2023-09-18
  Val:    150d  2023-09-26 ~ 2024-02-22
  Test:   150d  2024-03-01 ~ 2024-07-28

Fold 5 (walk_forward_rolling)
  Train:  800d  2021-12-08 ~ 2024-02-15
  Val:    150d  2024-02-23 ~ 2024-07-21
  Test:   150d  2024-07-29 ~ 2024-12-25

Fold 6 (walk_forward_rolling)
  T

Selected Features
DPO_20, btc_return_lag5, eth_btc_volume_ratio, eth_btc_corr_3d, sol_return, bnb_volume_change, GAP, ada_volume_ratio_20d, eth_btc_corr_7d, eth_intraday_range, sentiment_polarity, volume_lag2, INTRADAY_POSITION, btc_dominance, bnb_return, xrp_return, sentiment_volatility_7_lag1, btc_return_20d, price_percentile_250d, avax_volume_ratio_20d, l2_arbitrum_tvl_lag1, high_lag7, SMA_GOLDEN_CROSS, close_lag14, close_lag30, close_lag14_ratio, close_lag2, ATR_14, doge_volatility_30d, doge_return
Selected 30 features for this fold
Scaling completed for Fold 8

Saved raw data CSVs to model_results/2025-10-26/raw_data/direction/walk_forward

=== Fold 1 (walk_forward_rolling_reverse) ===
Fold 1 already completed. Loading results...
Loaded results for Fold 1

=== Fold 2 (walk_forward_rolling_reverse) ===
Fold 2 already completed. Loading results...
Loaded results for Fold 2

=== Fold 3 (walk_forward_rolling_reverse) ===
Fold 3 already completed. Loading results...
Loaded results for 

2025-10-27 09:00:44.600329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45865 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1f:00.0, compute capability: 8.6
2025-10-27 09:00:50.224427: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe8f9d44050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-27 09:00:50.224463: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-10-27 09:00:50.231620: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-27 09:00:50.680231: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1761523250.771710 3825656 device_compiler.h:186] Compiled cluster using XLA!  This line is 

[MLP] Train Acc: 0.7563 | Val Acc: 0.6267 | Gap: 0.1296
5/5 [==============================] - 0s 2ms/step
[LSTM] Train Loss: 1.2416 | Train Acc: 0.6260 | Val Loss: 1.2551 | Val Acc: 0.5833
4/4 [==============================] - 0s 3ms/step
[BiLSTM] Train Loss: 0.6710 | Train Acc: 0.6506 | Val Loss: 0.6831 | Val Acc: 0.6083
4/4 [==============================] - 0s 4ms/step
[GRU] Train Loss: 0.6829 | Train Acc: 0.6208 | Val Loss: 0.6871 | Val Acc: 0.6250
4/4 [==============================] - 0s 3ms/step
[TCN] Train Loss: 0.7135 | Train Acc: 0.6247 | Val Loss: 0.7459 | Val Acc: 0.5500
4/4 [==============================] - 0s 4ms/step
[CNN-LSTM] Train Loss: 0.6933 | Train Acc: 0.5026 | Val Loss: 0.6943 | Val Acc: 0.4750
4/4 [==============================] - 0s 3ms/step
[LSTM-Attention] Train Loss: 0.7163 | Train Acc: 0.5234 | Val Loss: 0.7174 | Val Acc: 0.5250
4/4 [==============================] - 0s 3ms/step
[DTW-LSTM] Train Loss: 0.6980 | Train Acc: 0.6039 | Val Loss: 0.7063 | Val 

[Voting Soft] Train Acc: 0.7700 | Val Acc: 0.6400 | Gap: 0.1300
[MLP] Train Acc: 0.7225 | Val Acc: 0.6467 | Gap: 0.0758
10/10 [==============================] - 0s 2ms/step
[LSTM] Train Loss: 0.8487 | Train Acc: 0.6545 | Val Loss: 0.8751 | Val Acc: 0.6083
9/9 [==============================] - 0s 3ms/step
[BiLSTM] Train Loss: 1.5827 | Train Acc: 0.5727 | Val Loss: 1.5885 | Val Acc: 0.5500
9/9 [==============================] - 0s 4ms/step
[GRU] Train Loss: 0.7609 | Train Acc: 0.6494 | Val Loss: 0.7847 | Val Acc: 0.6250
9/9 [==============================] - 0s 3ms/step
[TCN] Train Loss: 0.8289 | Train Acc: 0.6286 | Val Loss: 0.8537 | Val Acc: 0.6083
9/9 [==============================] - 0s 25ms/step
[CNN-LSTM] Train Loss: 0.6934 | Train Acc: 0.4987 | Val Loss: 0.6935 | Val Acc: 0.4583
9/9 [==============================] - 0s 12ms/step
[LSTM-Attention] Train Loss: 0.7251 | Train Acc: 0.5091 | Val Loss: 0.7274 | Val Acc: 0.4583
9/9 [==============================] - 0s 3ms/step
[DTW-LS